<font size="5">__NLP Final Project__</font><br><br>
<font size="3">__Topic Modeling: Finding Related Articles__</font><br><br>
Team Member:
- David Kurniadi
- Rene Lizarra
- Xin Gu

In [46]:
import numpy as np 
import pandas as pd
import os
import json
import glob
import sys
import scispacy
import spacy
import joblib
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from scipy.spatial.distance import jensenshannon
import pyLDAvis
import pyLDAvis.sklearn
from IPython.display import HTML, display
from ipywidgets import interact, Layout, HBox, VBox, Box
import ipywidgets as widgets
from IPython.display import clear_output
from tqdm import tqdm

import nltk
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
from sklearn.metrics.pairwise import cosine_similarity

In [47]:
import sys
import warnings

if not sys.warnoptions:
    warnings.simplefilter("ignore")

In [48]:
pyLDAvis.enable_notebook()

In [49]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', 100)

## Load and Prepare Data

### Kaggle_Covid19_All_Sources.csv

In [50]:
sys.path.insert(0, "../")
root_path = './CORD-19-research-challenge/2020-03-13/'

In [51]:
corona_features = {"doc_id": [None], "source": [None], "title": [None],
                  "abstract": [None], "text_body": [None]}
corona_df = pd.DataFrame.from_dict(corona_features)

In [52]:
corona_df.shape

(1, 5)

### Get all JSON files

In [53]:
json_filenames = glob.glob(f'{root_path}/**/*.json', recursive=True)

In [54]:
json_filenames

['./CORD-19-research-challenge/2020-03-13\\biorxiv_medrxiv\\0015023cc06b5362d332b3baf348d11567ca2fbb.json',
 './CORD-19-research-challenge/2020-03-13\\biorxiv_medrxiv\\004f0f8bb66cf446678dc13cf2701feec4f36d76.json',
 './CORD-19-research-challenge/2020-03-13\\biorxiv_medrxiv\\00d16927588fb04d4be0e6b269fc02f0d3c2aa7b.json',
 './CORD-19-research-challenge/2020-03-13\\biorxiv_medrxiv\\013d9d1cba8a54d5d3718c229b812d7cf91b6c89.json',
 './CORD-19-research-challenge/2020-03-13\\biorxiv_medrxiv\\01d162d7fae6aaba8e6e60e563ef4c2fca7b0e18.json',
 './CORD-19-research-challenge/2020-03-13\\biorxiv_medrxiv\\01e3b313e78a352593be2ff64927192af66619b5.json',
 './CORD-19-research-challenge/2020-03-13\\biorxiv_medrxiv\\02201e4601ab0eb70b6c26480cf2bfeae2625193.json',
 './CORD-19-research-challenge/2020-03-13\\biorxiv_medrxiv\\0255ea4b2f26a51a3bfa3bd8f3e1978c82c976d5.json',
 './CORD-19-research-challenge/2020-03-13\\biorxiv_medrxiv\\029c1c588047f1d612a219ee15494d2d19ff7439.json',
 './CORD-19-research-challen

### Iterate over the files and populate the data frame. 

In [55]:
################################################################################################
#
#    Title: Create Corona.csv File
#    Author: Frank Mitchell
#    Date: 2020
#    Code version: 1
#    Availability: https://www.kaggle.com/fmitchell259/create-corona-csv-file
# 
################################################################################################

def return_corona_df(json_filenames, df, source):

    for file_name in json_filenames:

        row = {"doc_id": None, "source": None, "title": None,
              "abstract": None, "text_body": None}

        with open(file_name) as json_data:
            data = json.load(json_data)

            row['doc_id'] = data['paper_id']
            row['title'] = data['metadata']['title']

            abstract_list = [data['abstract'][x]['text'] for x in range(len(data['abstract']) - 1)]
            abstract = "\n ".join(abstract_list)

            row['abstract'] = abstract
            
            body_list = []
            for _ in range(len(data['body_text'])):
                try:
                    body_list.append(data['body_text'][_]['text'])
                except:
                    pass

            body = "\n ".join(body_list)
            
            row['text_body'] = body
            
            if source == 'b':
                row['source'] = "biorxiv_medrxiv"
            elif source == "c":
                row['source'] = "common_use_sub"
            elif source == "n":
                row['source'] = "non_common_use"
            elif source == "p":
                row['source'] = "pmc_custom_license"
            
            df = df.append(row, ignore_index=True)
    
    return df

In [56]:
corona_df = return_corona_df(json_filenames, corona_df, 'b')

In [57]:
corona_df.shape

(13203, 5)

In [58]:
corona_out = corona_df.to_csv('kaggle_covid19.csv')

In [59]:
corona_df.head()

,doc_id,source,title,abstract,text_body
0,None,None,None,None,None
1,0015023cc06b5362d332b3baf348d11567ca2fbb,biorxiv_medrxiv,The RNA pseudoknots in foot-and-mouth disease virus are dispensable for genome replication but e...,word count: 194 22 Text word count: 5168 23 24 25 author/funder. All rights reserved. No reuse a...,"VP3, and VP0 (which is further processed to VP2 and VP4 during virus assembly) (6). The P2 64 an..."
2,004f0f8bb66cf446678dc13cf2701feec4f36d76,biorxiv_medrxiv,Healthcare-resource-adjusted vulnerabilities towards the 2019-nCoV epidemic across China,,"The 2019-nCoV epidemic has spread across China and 24 other countries 1-3 as of February 8, 2020..."
3,00d16927588fb04d4be0e6b269fc02f0d3c2aa7b,biorxiv_medrxiv,"Real-time, MinION-based, amplicon sequencing for lineage typing of infectious bronchitis virus f...",Infectious bronchitis (IB) causes significant economic losses in the global poultry industry. Co...,"Infectious bronchitis (IB), which is caused by infectious bronchitis virus (IBV), is one of the ..."
4,013d9d1cba8a54d5d3718c229b812d7cf91b6c89,biorxiv_medrxiv,"Assessing spread risk of Wuhan novel coronavirus within and beyond China, January-April 2020: a ...",,"In December 2019, a cluster of patients with pneumonia of unknown cause were reported in the cit..."


### Load All Sources Metadata

In [60]:
sources = pd.read_csv('./CORD-19-research-challenge/2020-03-13/all_sources_metadata_2020-03-13.csv')

sources.drop_duplicates(subset=['sha'], inplace=True)

def doi_url(d):
    if d.startswith('http://'):
        return d
    elif d.startswith('doi.org'):
        return f'http://{d}'
    else:
        return f'http://doi.org/{d}'

sources.doi = sources.doi.fillna('').apply(doi_url)

papers = pd.read_csv('kaggle_covid19.csv')
papers = papers.iloc[1:, 1:].reset_index(drop=True)

### Merge All Data Frames

In [61]:
cols_to_use = sources.columns.difference(papers.columns)
all_data = pd.merge(papers, sources[cols_to_use], left_on='doc_id', right_on='sha', how='left')

In [62]:
all_data.to_csv('Kaggle_Covid19_All_Sources.csv')

### Upload from CSV into Data Frame for building LDA model

In [63]:
all_data = pd.read_csv('Kaggle_Covid19_All_Sources.csv')

### Data Preparations

In [64]:
all_data['publish_year'] = all_data.publish_time.str[:4].fillna(-1).astype(int) # 360 times None

all_data.title = all_data.title.astype(str) # change to string, there are also some numeric values

In [65]:
all_data.head(2)

,Unnamed: 0,doc_id,source,title,abstract,text_body,Microsoft Academic Paper ID,WHO #Covidence,authors,doi,has_full_text,journal,license,pmcid,publish_time,pubmed_id,sha,source_x,publish_year
0,0,0015023cc06b5362d332b3baf348d11567ca2fbb,biorxiv_medrxiv,The RNA pseudoknots in foot-and-mouth disease virus are dispensable for genome replication but e...,word count: 194 22 Text word count: 5168 23 24 25 author/funder. All rights reserved. No reuse a...,"VP3, and VP0 (which is further processed to VP2 and VP4 during virus assembly) (6). The P2 64 an...",NaN,NaN,"Ward, J. C. J.; Lasecka-Dykes, L.; Neil, C.; Adeyemi, O.; Gold, S.; McLean, N.; Wright, C.; Hero...",http://doi.org/10.1101/2020.01.10.901801,True,NaN,See https://www.biorxiv.org/about-biorxiv,NaN,2020-01-11,NaN,0015023cc06b5362d332b3baf348d11567ca2fbb,biorxiv,2020
1,1,004f0f8bb66cf446678dc13cf2701feec4f36d76,biorxiv_medrxiv,Healthcare-resource-adjusted vulnerabilities towards the 2019-nCoV epidemic across China,NaN,"The 2019-nCoV epidemic has spread across China and 24 other countries 1-3 as of February 8, 2020...",NaN,NaN,NaN,http://doi.org/10.1101/2020.02.11.20022111,True,NaN,See https://www.medrxiv.org/submit-a-manuscript,NaN,NaN,NaN,004f0f8bb66cf446678dc13cf2701feec4f36d76,medrxiv,-1


### We consider the text body, but the approach could also be applied to the abstracts only.

In [66]:
all_texts = all_data.text_body

In [67]:
all_texts[0][:500]

'VP3, and VP0 (which is further processed to VP2 and VP4 during virus assembly) (6). The P2 64 and P3 regions encode the non-structural proteins 2B and 2C and 3A, 3B (1-3) (VPg), 3C pro and 4 structural protein-coding region is replaced by reporter genes, allow the study of genome 68 replication without the requirement for high containment (9, 10) ( figure 1A ).\n The FMDV 5′ UTR is the largest known picornavirus UTR, comprising approximately 1300 71 nucleotides and containing several highly struc'

## Latend Dirichlet Allocation

### Medium Model

In [68]:
import en_core_sci_md
nlp = en_core_sci_md.load(disable=["tagger", "parser", "ner"])
nlp.max_length = 2000000

In [69]:
#########################################################################################################################
#
#    Title: Topic Modeling: Finding Related Articles
#    Author: Daniel Wolffram
#    Date: 2020
#    Code version: 10
#    Availability: https://www.kaggle.com/danielwolffram/topic-modeling-finding-related-articles?scriptVersionId=30463507
# 
#########################################################################################################################
def spacy_tokenizer(sentence):
    return [word.lemma_ for word in nlp(sentence) if not (word.like_num or word.is_stop or word.is_punct or word.is_space)] # remove numbers (e.g. from references [1], etc.)

def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        message = "\nTopic #%d: " % topic_idx
        message += " ".join([feature_names[i]
                             for i in topic.argsort()[:-n_top_words - 1:-1]])
        print(message)
    print()

### New stop words list 

In [70]:
#########################################################################################################################
#
#    Title: Topic Modeling: Finding Related Articles
#    Author: Daniel Wolffram
#    Date: 2020
#    Code version: 10
#    Availability: https://www.kaggle.com/danielwolffram/topic-modeling-finding-related-articles?scriptVersionId=30463507
# 
#########################################################################################################################
customize_stop_words = [
    'doi', 'preprint', 'copyright', 'peer', 'reviewed', 'org', 'https', 'et', 'al', 'author', 'figure', 
    'rights', 'reserved', 'permission', 'used', 'using', 'biorxiv', 'fig', 'fig.', 'al.',
    'di', 'la', 'il', 'del', 'le', 'della', 'dei', 'delle', 'una', 'da',  'dell',  'non', 'si'
]

for w in customize_stop_words:
    nlp.vocab[w].is_stop = True

In [71]:
tf_vectorizer = CountVectorizer(tokenizer = spacy_tokenizer)

tf = tf_vectorizer.fit_transform(tqdm(all_texts))

tf.shape

100%|████████████████████████████████████████████████████████████████████████████| 13202/13202 [05:53<00:00, 37.34it/s]


(13202, 641818)

In [72]:
joblib.dump(tf_vectorizer, 'tf_vectorizer.csv')
joblib.dump(tf, 'tf.csv')

['tf.csv']

In [73]:
lda_tf = LatentDirichletAllocation(n_components=50, random_state=0, n_jobs=-1)
lda_tf.fit(tf)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=50, n_jobs=-1,
                          perp_tol=0.1, random_state=0, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

In [74]:
joblib.dump(lda_tf, 'lda.csv')

['lda.csv']

In [75]:
joblib.dump(lda_tf, 'lda.pkl')

['lda.pkl']

In [76]:
lda_tf = joblib.load('lda.pkl')

## Discovered Topics

In [77]:
tfidf_feature_names = tf_vectorizer.get_feature_names()
print_top_words(lda_tf, tfidf_feature_names, 25)


Topic #0: disease increase cell study effect lung cancer cause include patient tissue lesion show treatment clinical report associate find inflammation bacterium inflammatory image damage activity reduce

Topic #1: q prp cell ifit1 show datum sc resistance increase bind observe mm model line week numb antibiotic prion concentration ifit3 result complex time growth license

Topic #2: cell virus viral hiv-1 infection entry hiv fusion ifitm3 virion protein particle receptor target membrane human env antiviral t replication host inhibit glycoprotein infect envelope

Topic #3: ace2 ang ii activity ace heart increase diabetes level mouse kidney cardiac facemask effect ras receptor study renal ang-(1 plasma cardiovascular hypertension expression angiotensin inhibitor

Topic #4: cat sample fcov fip t. study c type result fipv ° show p. antibody control gondii high temperature test table min extract feline fee time

Topic #5: dog cat = study p group disease concentration clinical test sample h

Topic #48: disease human health pathogen animal infectious country population global risk emerge new control research include increase cause infection area world example development approach africa change

Topic #49: cell expression gene protein infection response level prrsv express signal show study ifn analysis pathway result immune control type virus antibody induce h c datum



### Visualization

In [78]:
#########################################################################################################################
#
#    Title: Topic Modeling: Finding Related Articles
#    Author: Daniel Wolffram
#    Date: 2020
#    Code version: 10
#    Availability: https://www.kaggle.com/danielwolffram/topic-modeling-finding-related-articles?scriptVersionId=30463507
# 
#########################################################################################################################
viz = pyLDAvis.sklearn.prepare(lda_tf, tf, tf_vectorizer)
pyLDAvis.display(viz)
pyLDAvis.save_html(viz, 'lda.html')

In [79]:
topic_dist = pd.DataFrame(lda_tf.transform(tf))

In [80]:
topic_dist.to_csv('topic_dist.csv', index=False)

In [81]:
topic_dist.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49
0,0.000025,0.000025,0.000025,0.000025,0.000025,0.000025,0.000025,0.000025,0.000025,0.000025,0.000025,0.000025,0.000025,0.000025,0.259887,0.000025,0.000025,0.000025,0.000025,0.000025,0.000025,0.000025,0.000025,0.000025,0.000025,0.000025,0.000025,0.000025,0.000025,0.000025,0.000025,0.000025,0.000025,0.000025,0.000025,0.000025,0.000025,0.000025,0.442560,0.000025,0.000025,0.064784,0.231597,0.000025,0.000025,0.000025,0.000025,0.000025,0.000025,0.000025
1,0.000050,0.000050,0.000050,0.000050,0.000050,0.000050,0.000050,0.000050,0.000050,0.000050,0.000050,0.000050,0.000050,0.038465,0.000050,0.133305,0.000050,0.000050,0.000050,0.000050,0.000050,0.000050,0.012764,0.784700,0.000050,0.000050,0.000050,0.000050,0.000050,0.000050,0.000050,0.000050,0.000050,0.000050,0.000050,0.000050,0.000050,0.000050,0.000050,0.000050,0.000050,0.000050,0.028499,0.000050,0.000050,0.000050,0.000050,0.000050,0.000050,0.000050
2,0.000010,0.000010,0.000010,0.000010,0.000010,0.000010,0.000010,0.000010,0.581407,0.000010,0.000010,0.000010,0.000010,0.000010,0.000010,0.000010,0.000010,0.000010,0.000010,0.000010,0.000010,0.000010,0.000010,0.000010,0.000010,0.000010,0.000010,0.000010,0.000010,0.000010,0.000010,0.000010,0.000010,0.000010,0.000010,0.000010,0.003909,0.000010,0.000010,0.000010,0.000010,0.355169,0.059073,0.000010,0.000010,0.000010,0.000010,0.000010,0.000010,0.000010
3,0.000008,0.000008,0.000008,0.000008,0.000008,0.000008,0.000008,0.000008,0.000008,0.000008,0.000008,0.000008,0.000008,0.067244,0.000008,0.000008,0.000008,0.000008,0.000008,0.000008,0.000008,0.000008,0.000008,0.797320,0.000008,0.000008,0.000008,0.000008,0.000008,0.000008,0.000008,0.000008,0.000008,0.000008,0.000008,0.000008,0.000008,0.000008,0.000008,0.000008,0.000008,0.000008,0.135038,0.000008,0.000008,0.000008,0.000008,0.000008,0.000008,0.000008
4,0.005689,0.000009,0.020226,0.043477,0.000009,0.000009,0.000009,0.000009,0.000009,0.000009,0.000009,0.000009,0.000009,0.000009,0.000009,0.090508,0.000009,0.000009,0.317526,0.000009,0.135642,0.063361,0.029706,0.000009,0.032777,0.000009,0.000009,0.091177,0.000009,0.000009,0.000009,0.000009,0.007241,0.000009,0.000009,0.000009,0.000009,0.000009,0.000009,0.000009,0.000009,0.000009,0.162339,0.000009,0.000009,0.000009,0.000009,0.000009,0.000009,0.000009


In [82]:
topic_dist.shape

(13202, 50)

## Get "Nearest" Papers (in Topic Space)

In [83]:
#########################################################################################################################
#
#    Title: Topic Modeling: Finding Related Articles
#    Author: Daniel Wolffram
#    Date: 2020
#    Code version: 10
#    Availability: https://www.kaggle.com/danielwolffram/topic-modeling-finding-related-articles?scriptVersionId=30463507
# 
#########################################################################################################################
def get_k_nearest_docs(doc_dist, k=5, lower=1950, upper=2020, only_covid19=False):
    '''
    doc_dist: topic distribution (sums to 1) of one article
    
    Returns the index of the k nearest articles (as by Jensen–Shannon divergence in topic space). 
    '''
    
    relevant_time = all_data.publish_year.between(lower, upper)
    
    if only_covid19:
        is_covid19_article = all_data.text_body.str.contains('COVID-19|SARS-CoV-2|2019-nCov')
        topic_dist_temp = topic_dist[relevant_time & is_covid19_article]
        
    else:
        topic_dist_temp = topic_dist[relevant_time]
         
    distances = topic_dist_temp.apply(lambda x: jensenshannon(x, doc_dist), axis=1)
    k_nearest = distances[distances != 0].nsmallest(n=k).index
        
    return k_nearest

## Search related papers to a chosen one

In [84]:
#########################################################################################################################
#
#    Title: Topic Modeling: Finding Related Articles
#    Author: Daniel Wolffram
#    Date: 2020
#    Code version: 10
#    Availability: https://www.kaggle.com/danielwolffram/topic-modeling-finding-related-articles?scriptVersionId=30463507
# 
#########################################################################################################################
def recommendation(doc_id, k=5, lower=1950, upper=2020, only_covid19=False):
    '''
    Returns the title of the k papers that are closest (topic-wise) to the paper given by paper_id.
    '''
    
    print(all_data.title[all_data.doc_id == doc_id].values[0])

    recommended = get_k_nearest_docs(topic_dist[all_data.doc_id == doc_id].iloc[0], k, lower, upper, only_covid19)
    recommended = all_data.iloc[recommended]
    
    h = '<br/>'.join(['<a href="' + l + '" target="_blank">'+ n + '</a>' for l, n in recommended[['doi','title']].values])
    display(HTML(h))

In [85]:
recommendation('a137eb51461b4a4ed3980aa5b9cb2f2c1cf0292a', k=5, lower=2005, upper=2018, only_covid19=False)

The effect of inhibition of PP1 and TNFα signaling on pathogenesis of SARS coronavirus


In [86]:
recommendation('a137eb51461b4a4ed3980aa5b9cb2f2c1cf0292a', k=5, lower=1950, upper=2020, only_covid19=True)

The effect of inhibition of PP1 and TNFα signaling on pathogenesis of SARS coronavirus


In [87]:
recommendation('90b5ecf991032f3918ad43b252e17d1171b4ea63', k=5, only_covid19=True)

The role of absolute humidity on transmission rates of the COVID-19 outbreak


In [88]:
recommendation('c04c7fb330a409a00f67040dde0f83b3da88eacb', k=5, only_covid19=True)

Potential inhibitors for 2019-nCoV coronavirus M protease from clinically approved medicines


In [89]:
recommendation('36521caf90f471c9da1a4e84f8562440d73ead9a', k=10)

Estimation of the epidemic properties of the 2019 novel coronavirus: A mathematical modeling study


## Widget: Pick a COVID-19-Paper

In [90]:
#########################################################################################################################
#
#    Title: Topic Modeling: Finding Related Articles
#    Author: Daniel Wolffram
#    Date: 2020
#    Code version: 10
#    Availability: https://www.kaggle.com/danielwolffram/topic-modeling-finding-related-articles?scriptVersionId=30463507
# 
#########################################################################################################################
def related_papers():
    '''
    Creates a widget where you can select one of many papers about covid-19 and then displays related articles from the whole dataset.
    '''
    covid_papers = all_data[all_data.text_body.str.contains('COVID-19|SARS-CoV-2|2019-nCov')][['doc_id', 'title']] # are there more names?
    title_to_id = covid_papers.set_index('title')['doc_id'].to_dict()
    
    def main_function(bullet, k=5, year_range=[1950, 2020], only_covid19=False):
        recommendation(title_to_id[bullet], k, lower=year_range[0], upper=year_range[1], only_covid19=only_covid19)
    
    yearW = widgets.IntRangeSlider(min=1950, max=2020, value=[2010, 2020], description='Year Range', 
                                   continuous_update=False, layout=Layout(width='40%'))
    covidW = widgets.Checkbox(value=False,description='Only COVID-19-Papers',disabled=False, indent=False, layout=Layout(width='20%'))
    kWidget = widgets.IntSlider(value=10, description='k', max=50, min=1, layout=Layout(width='20%'))

    bulletW = widgets.Select(options=title_to_id.keys(), layout=Layout(width='90%', height='200px'), description='Title:')

    widget = widgets.interactive(main_function, bullet=bulletW, k=kWidget, year_range=yearW, only_covid19=covidW)

    controls = VBox([Box(children=[widget.children[:-1][1], widget.children[:-1][2], widget.children[:-1][3]], 
                         layout=Layout(justify_content='space-around')), widget.children[:-1][0]])
    output = widget.children[-1]
    display(VBox([controls, output]))

In [91]:
related_papers()

## Browse Tasks

We can now also map a task or bullet point into the topic space and find related articles that might help to solve the question at hand.

Note: Some of the bullet points are very short - results might not be reliable in this case.

(A similar approach, but with a different underlying model, can be found in CORD-19 Search articles with Doc2Vec)

In [92]:
#########################################################################################################################
#
#    Title: Topic Modeling: Finding Related Articles
#    Author: Daniel Wolffram
#    Date: 2020
#    Code version: 10
#    Availability: https://www.kaggle.com/danielwolffram/topic-modeling-finding-related-articles?scriptVersionId=30463507
# 
#########################################################################################################################
task1 = ["Range of incubation periods for the disease in humans (and how this varies across age and health status) and how long individuals are contagious, even after recovery.",
"Prevalence of asymptomatic shedding and transmission (e.g., particularly children).",
"Seasonality of transmission.",
"Physical science of the coronavirus (e.g., charge distribution, adhesion to hydrophilic/phobic surfaces, environmental survival to inform decontamination efforts for affected areas and provide information about viral shedding).",
"Persistence and stability on a multitude of substrates and sources (e.g., nasal discharge, sputum, urine, fecal matter, blood).",
"Persistence of virus on surfaces of different materials (e,g., copper, stainless steel, plastic).",
"Natural history of the virus and shedding of it from an infected person",
"Implementation of diagnostics and products to improve clinical processes",
"Disease models, including animal models for infection, disease and transmission",
"Tools and studies to monitor phenotypic change and potential adaptation of the virus",
"Immune response and immunity",
"Effectiveness of movement control strategies to prevent secondary transmission in health care and community settings",
 "Effectiveness of personal protective equipment (PPE) and its usefulness to reduce risk of transmission in health care and community settings",
"Role of the environment in transmission"]

task2 = ['Data on potential risks factors',
'Smoking, pre-existing pulmonary disease',
'Co-infections (determine whether co-existing respiratory/viral infections make the virus more transmissible or virulent) and other co-morbidities',
'Neonates and pregnant women',
'Socio-economic and behavioral factors to understand the economic impact of the virus and whether there were differences.',
'Transmission dynamics of the virus, including the basic reproductive number, incubation period, serial interval, modes of transmission and environmental factors', 
'Severity of disease, including risk of fatality among symptomatic hospitalized patients, and high-risk patient groups',
'Susceptibility of populations',
'Public health mitigation measures that could be effective for control']

task3 = ['Real-time tracking of whole genomes and a mechanism for coordinating the rapid dissemination of that information to inform the development of diagnostics and therapeutics and to track variations of the virus over time.',
'Access to geographic and temporal diverse sample sets to understand geographic distribution and genomic differences, and determine whether there is more than one strain in circulation. Multi-lateral agreements such as the Nagoya Protocol could be leveraged.',
'Evidence that livestock could be infected (e.g., field surveillance, genetic sequencing, receptor binding) and serve as a reservoir after the epidemic appears to be over.',
'Evidence of whether farmers are infected, and whether farmers could have played a role in the origin.',
'Surveillance of mixed wildlife- livestock farms for SARS-CoV-2 and other coronaviruses in Southeast Asia.',
'Experimental infections to test host range for this pathogen.',
'Animal host(s) and any evidence of continued spill-over to humans',
'Socioeconomic and behavioral risk factors for this spill-over',
'Sustainable risk reduction strategies']

task4 = ["Guidance on ways to scale up NPIs in a more coordinated way (e.g., establish funding, infrastructure and authorities to support real time, authoritative (qualified participants) collaboration with all states to gain consensus on consistent guidance and to mobilize resources to geographic areas where critical shortfalls are identified) to give us time to enhance our health care delivery system capacity to respond to an increase in cases.",
"Rapid design and execution of experiments to examine and compare NPIs currently being implemented. DHS Centers for Excellence could potentially be leveraged to conduct these experiments.",
"Rapid assessment of the likely efficacy of school closures, travel bans, bans on mass gatherings of various sizes, and other social distancing approaches.",
"Methods to control the spread in communities, barriers to compliance and how these vary among different populations..",
"Models of potential interventions to predict costs and benefits that take account of such factors as race, income, disability, age, geographic location, immigration status, housing status, employment status, and health insurance status.",
"Policy changes necessary to enable the compliance of individuals with limited resources and the underserved with NPIs.",
"Research on why people fail to comply with public health advice, even if they want to do so (e.g., social or financial costs may be too high).",
"Research on the economic impact of this or any pandemic. This would include identifying policy and programmatic alternatives that lessen/mitigate risks to critical government services, food distribution and supplies, access to critical household supplies, and access to health diagnoses, treatment, and needed care, regardless of ability to pay."]

task5 = ["Effectiveness of drugs being developed and tried to treat COVID-19 patients. Clinical and bench trials to investigate less common viral inhibitors against COVID-19 such as naproxen, clarithromycin, and minocyclinethat that may exert effects on viral replication.",
"Methods evaluating potential complication of Antibody-Dependent Enhancement (ADE) in vaccine recipients.",
"Exploration of use of best animal models and their predictive value for a human vaccine.",
"Capabilities to discover a therapeutic (not vaccine) for the disease, and clinical effectiveness studies to discover therapeutics, to include antiviral agents.",
"Alternative models to aid decision makers in determining how to prioritize and distribute scarce, newly proven therapeutics as production ramps up. This could include identifying approaches for expanding production capacity to ensure equitable and timely distribution to populations in need.",
"Efforts targeted at a universal coronavirus vaccine.",
"Efforts to develop animal models and standardize challenge studies",
"Efforts to develop prophylaxis clinical studies and prioritize in healthcare workers",
"Approaches to evaluate risk for enhanced disease after vaccination",
"Assays to evaluate vaccine immune response and process development for vaccines, alongside suitable animal models [in conjunction with therapeutics]"]

task6 = ["Efforts to articulate and translate existing ethical principles and standards to salient issues in COVID-2019", 
"Efforts to embed ethics across all thematic areas, engage with novel ethical issues that arise and coordinate to minimize duplication of oversight",
"Efforts to support sustained education, access, and capacity building in the area of ethics",
"Efforts to establish a team at WHO that will be integrated within multidisciplinary research and operational platforms and that will connect with existing and expanded global networks of social sciences.",
"Efforts to develop qualitative assessment frameworks to systematically collect information related to local barriers and enablers for the uptake and adherence to public health measures for prevention and control. This includes the rapid identification of the secondary impacts of these measures. (e.g. use of surgical masks, modification of health seeking behaviors for SRH, school closures)",
"Efforts to identify how the burden of responding to the outbreak and implementing public health measures affects the physical and psychological health of those providing care for Covid-19 patients and identify the immediate needs that must be addressed.",
"Efforts to identify the underlying drivers of fear, anxiety and stigma that fuel misinformation and rumor, particularly through social media."]

task7 = ["How widespread current exposure is to be able to make immediate policy recommendations on mitigation measures. Denominators for testing and a mechanism for rapidly sharing that information, including demographics, to the extent possible. Sampling methods to determine asymptomatic disease (e.g., use of serosurveys (such as convalescent samples) and early detection of disease (e.g., use of screening of neutralizing antibodies such as ELISAs).",
"Efforts to increase capacity on existing diagnostic platforms and tap into existing surveillance platforms.",
"Recruitment, support, and coordination of local expertise and capacity (public, private—commercial, and non-profit, including academic), including legal, ethical, communications, and operational issues.",
"National guidance and guidelines about best practices to states (e.g., how states might leverage universities and private laboratories for testing purposes, communications to public health officials and the public).",
"Development of a point-of-care test (like a rapid influenza test) and rapid bed-side tests, recognizing the tradeoffs between speed, accessibility, and accuracy.",
"Rapid design and execution of targeted surveillance experiments calling for all potential testers using PCR in a defined area to start testing and report to a specific entity. These experiments could aid in collecting longitudinal samples, which are critical to understanding the impact of ad hoc local interventions (which also need to be recorded).",
"Separation of assay development issues from instruments, and the role of the private sector to help quickly migrate assays onto those devices.",
"Efforts to track the evolution of the virus (i.e., genetic drift or mutations) and avoid locking into specific reagents and surveillance/detection schemes.",
"Latency issues and when there is sufficient viral load to detect the pathogen, and understanding of what is needed in terms of biological and environmental sampling.",
"Use of diagnostics such as host response markers (e.g., cytokines) to detect early disease or predict severe disease progression, which would be important to understanding best clinical practice and efficacy of therapeutic interventions.",
"Policies and protocols for screening and testing.",
"Policies to mitigate the effects on supplies associated with mass testing, including swabs and reagents.",
"Technology roadmap for diagnostics.",
"Barriers to developing and scaling up new diagnostic tests (e.g., market forces), how future coalition and accelerator models (e.g., Coalition for Epidemic Preparedness Innovations) could provide critical funding for diagnostics, and opportunities for a streamlined regulatory environment.",
"New platforms and technology (e.g., CRISPR) to improve response times and employ more holistic approaches to COVID-19 and future diseases.",
"Coupling genomics and diagnostic testing on a large scale.",
"Enhance capabilities for rapid sequencing and bioinformatics to target regions of the genome that will allow specificity for a particular variant.",
"Enhance capacity (people, technology, data) for sequencing with advanced analytics for unknown pathogens, and explore capabilities for distinguishing naturally-occurring pathogens from intentional.",
"One Health surveillance of humans and potential sources of future spillover or ongoing exposure for this organism and future pathogens, including both evolutionary hosts (e.g., bats) and transmission hosts (e.g., heavily trafficked and farmed wildlife and domestic food and companion species), inclusive of environmental, demographic, and occupational risk factors."]

task8 = ["Resources to support skilled nursing facilities and long term care facilities.",
"Mobilization of surge medical staff to address shortages in overwhelmed communities",
"Age-adjusted mortality data for Acute Respiratory Distress Syndrome (ARDS) with/without other organ failure – particularly for viral etiologies",
"Extracorporeal membrane oxygenation (ECMO) outcomes data of COVID-19 patients",
"Outcomes data for COVID-19 after mechanical ventilation adjusted for age.",
"Knowledge of the frequency, manifestations, and course of extrapulmonary manifestations of COVID-19, including, but not limited to, possible cardiomyopathy and cardiac arrest.",
"Application of regulatory standards (e.g., EUA, CLIA) and ability to adapt care to crisis standards of care level.",
"Approaches for encouraging and facilitating the production of elastomeric respirators, which can save thousands of N95 masks.",
"Best telemedicine practices, barriers and faciitators, and specific actions to remove/expand them within and across state boundaries.",
"Guidance on the simple things people can do at home to take care of sick people and manage disease.",
"Oral medications that might potentially work.",
"Use of AI in real-time health care delivery to evaluate interventions, risk factors, and outcomes in a way that could not be done manually.",
"Best practices and critical challenges and innovative solutions and technologies in hospital flow and organization, workforce protection, workforce allocation, community-based support resources, payment, and supply chain management to enhance capacity, efficiency, and outcomes.",
"Efforts to define the natural history of disease to inform clinical care, public health interventions, infection prevention control, transmission, and clinical trials",
"Efforts to develop a core clinical outcome set to maximize usability of data across a range of trials",
"Efforts to determine adjunctive and supportive interventions that can improve the clinical outcomes of infected patients (e.g. steroids, high flow oxygen)"]

task9 = ["Methods for coordinating data-gathering with standardized nomenclature.",
"Sharing response information among planners, providers, and others.",
"Understanding and mitigating barriers to information-sharing.",
"How to recruit, support, and coordinate local (non-Federal) expertise and capacity relevant to public health emergency response (public, private, commercial and non-profit, including academic).",
"Integration of federal/state/local public health surveillance systems.",
"Value of investments in baseline public health response infrastructure preparedness",
"Modes of communicating with target high-risk populations (elderly, health care workers).",
"Risk communication and guidelines that are easy to understand and follow (include targeting at risk populations’ families too).",
"Communication that indicates potential risk of disease to all population groups.",
"Misunderstanding around containment and mitigation.",
"Action plan to mitigate gaps and problems of inequity in the Nation’s public health capability, capacity, and funding to ensure all citizens in need are supported and can access information, surveillance, and treatment.",
"Measures to reach marginalized and disadvantaged populations.",
"Data systems and research priorities and agendas incorporate attention to the needs and circumstances of disadvantaged populations and underrepresented minorities.",
"Mitigating threats to incarcerated people from COVID-19, assuring access to information, prevention, diagnosis, and treatment.",
"Understanding coverage policies (barriers and opportunities) related to testing, treatment, and care"]

In [93]:
#########################################################################################################################
#
#    Title: Topic Modeling: Finding Related Articles
#    Author: Daniel Wolffram
#    Date: 2020
#    Code version: 10
#    Availability: https://www.kaggle.com/danielwolffram/topic-modeling-finding-related-articles?scriptVersionId=30463507
# 
#########################################################################################################################
def relevant_articles(tasks, k=3, lower=1950, upper=2020, only_covid19=False):
    tasks = [tasks] if type(tasks) is str else tasks 
    
    tasks_tf = tf_vectorizer.transform(tasks)
    tasks_topic_dist = pd.DataFrame(lda_tf.transform(tasks_tf))

    for index, bullet in enumerate(tasks):
        print(bullet)
        recommended = get_k_nearest_docs(tasks_topic_dist.iloc[index], k, lower, upper, only_covid19)
        recommended = all_data.iloc[recommended]

        h = '<br/>'.join(['<a href="' + l + '" target="_blank">'+ n + '</a>' for l, n in recommended[['doi','title']].values])
        display(HTML(h))

### What is known about transmission, incubation, and environmental stability?

In [94]:
relevant_articles(task1, 5, only_covid19=True)

Range of incubation periods for the disease in humans (and how this varies across age and health status) and how long individuals are contagious, even after recovery.


Prevalence of asymptomatic shedding and transmission (e.g., particularly children).


Seasonality of transmission.


Physical science of the coronavirus (e.g., charge distribution, adhesion to hydrophilic/phobic surfaces, environmental survival to inform decontamination efforts for affected areas and provide information about viral shedding).


Persistence and stability on a multitude of substrates and sources (e.g., nasal discharge, sputum, urine, fecal matter, blood).


...

Tools and studies to monitor phenotypic change and potential adaptation of the virus


Immune response and immunity


Effectiveness of movement control strategies to prevent secondary transmission in health care and community settings


Effectiveness of personal protective equipment (PPE) and its usefulness to reduce risk of transmission in health care and community settings


Role of the environment in transmission


### What do we know about COVID-19 risk factors?

In [95]:
relevant_articles(task2, 5, only_covid19=True)

Data on potential risks factors


Smoking, pre-existing pulmonary disease


Co-infections (determine whether co-existing respiratory/viral infections make the virus more transmissible or virulent) and other co-morbidities


Neonates and pregnant women


Socio-economic and behavioral factors to understand the economic impact of the virus and whether there were differences.


Transmission dynamics of the virus, including the basic reproductive number, incubation period, serial interval, modes of transmission and environmental factors


Severity of disease, including risk of fatality among symptomatic hospitalized patients, and high-risk patient groups


Susceptibility of populations


Public health mitigation measures that could be effective for control


### What do we know about virus genetics, origin, and evolution?

In [96]:
relevant_articles(task3, 5, only_covid19=True)

Real-time tracking of whole genomes and a mechanism for coordinating the rapid dissemination of that information to inform the development of diagnostics and therapeutics and to track variations of the virus over time.


Access to geographic and temporal diverse sample sets to understand geographic distribution and genomic differences, and determine whether there is more than one strain in circulation. Multi-lateral agreements such as the Nagoya Protocol could be leveraged.


Evidence that livestock could be infected (e.g., field surveillance, genetic sequencing, receptor binding) and serve as a reservoir after the epidemic appears to be over.


Evidence of whether farmers are infected, and whether farmers could have played a role in the origin.


Surveillance of mixed wildlife- livestock farms for SARS-CoV-2 and other coronaviruses in Southeast Asia.


Experimental infections to test host range for this pathogen.


Animal host(s) and any evidence of continued spill-over to humans


Socioeconomic and behavioral risk factors for this spill-over


Sustainable risk reduction strategies


### What do we know about non-pharmaceutical interventions?

In [97]:
relevant_articles(task4, 5, only_covid19=True)

Guidance on ways to scale up NPIs in a more coordinated way (e.g., establish funding, infrastructure and authorities to support real time, authoritative (qualified participants) collaboration with all states to gain consensus on consistent guidance and to mobilize resources to geographic areas where critical shortfalls are identified) to give us time to enhance our health care delivery system capacity to respond to an increase in cases.


Rapid design and execution of experiments to examine and compare NPIs currently being implemented. DHS Centers for Excellence could potentially be leveraged to conduct these experiments.


Rapid assessment of the likely efficacy of school closures, travel bans, bans on mass gatherings of various sizes, and other social distancing approaches.


Methods to control the spread in communities, barriers to compliance and how these vary among different populations..


Models of potential interventions to predict costs and benefits that take account of such factors as race, income, disability, age, geographic location, immigration status, housing status, employment status, and health insurance status.


Policy changes necessary to enable the compliance of individuals with limited resources and the underserved with NPIs.


Research on why people fail to comply with public health advice, even if they want to do so (e.g., social or financial costs may be too high).


Research on the economic impact of this or any pandemic. This would include identifying policy and programmatic alternatives that lessen/mitigate risks to critical government services, food distribution and supplies, access to critical household supplies, and access to health diagnoses, treatment, and needed care, regardless of ability to pay.


### What do we know about vaccines and therapeutics?

In [98]:
relevant_articles(task5, 5, only_covid19=True)

Effectiveness of drugs being developed and tried to treat COVID-19 patients. Clinical and bench trials to investigate less common viral inhibitors against COVID-19 such as naproxen, clarithromycin, and minocyclinethat that may exert effects on viral replication.


Methods evaluating potential complication of Antibody-Dependent Enhancement (ADE) in vaccine recipients.


Exploration of use of best animal models and their predictive value for a human vaccine.


Capabilities to discover a therapeutic (not vaccine) for the disease, and clinical effectiveness studies to discover therapeutics, to include antiviral agents.


Alternative models to aid decision makers in determining how to prioritize and distribute scarce, newly proven therapeutics as production ramps up. This could include identifying approaches for expanding production capacity to ensure equitable and timely distribution to populations in need.


Efforts targeted at a universal coronavirus vaccine.


Efforts to develop animal models and standardize challenge studies


Efforts to develop prophylaxis clinical studies and prioritize in healthcare workers


Approaches to evaluate risk for enhanced disease after vaccination


Assays to evaluate vaccine immune response and process development for vaccines, alongside suitable animal models [in conjunction with therapeutics]


### What has been published about ethical and social science considerations?

In [99]:
relevant_articles(task6, 5, only_covid19=True)

Efforts to articulate and translate existing ethical principles and standards to salient issues in COVID-2019


Efforts to embed ethics across all thematic areas, engage with novel ethical issues that arise and coordinate to minimize duplication of oversight


Efforts to support sustained education, access, and capacity building in the area of ethics


Efforts to establish a team at WHO that will be integrated within multidisciplinary research and operational platforms and that will connect with existing and expanded global networks of social sciences.


Efforts to develop qualitative assessment frameworks to systematically collect information related to local barriers and enablers for the uptake and adherence to public health measures for prevention and control. This includes the rapid identification of the secondary impacts of these measures. (e.g. use of surgical masks, modification of health seeking behaviors for SRH, school closures)


Efforts to identify how the burden of responding to the outbreak and implementing public health measures affects the physical and psychological health of those providing care for Covid-19 patients and identify the immediate needs that must be addressed.


Efforts to identify the underlying drivers of fear, anxiety and stigma that fuel misinformation and rumor, particularly through social media.


### What do we know about diagnostics and surveillance?

In [100]:
relevant_articles(task7, 5, only_covid19=True)

How widespread current exposure is to be able to make immediate policy recommendations on mitigation measures. Denominators for testing and a mechanism for rapidly sharing that information, including demographics, to the extent possible. Sampling methods to determine asymptomatic disease (e.g., use of serosurveys (such as convalescent samples) and early detection of disease (e.g., use of screening of neutralizing antibodies such as ELISAs).


Efforts to increase capacity on existing diagnostic platforms and tap into existing surveillance platforms.


Recruitment, support, and coordination of local expertise and capacity (public, private—commercial, and non-profit, including academic), including legal, ethical, communications, and operational issues.


National guidance and guidelines about best practices to states (e.g., how states might leverage universities and private laboratories for testing purposes, communications to public health officials and the public).


Development of a point-of-care test (like a rapid influenza test) and rapid bed-side tests, recognizing the tradeoffs between speed, accessibility, and accuracy.


...

New platforms and technology (e.g., CRISPR) to improve response times and employ more holistic approaches to COVID-19 and future diseases.


Coupling genomics and diagnostic testing on a large scale.


Enhance capabilities for rapid sequencing and bioinformatics to target regions of the genome that will allow specificity for a particular variant.


Enhance capacity (people, technology, data) for sequencing with advanced analytics for unknown pathogens, and explore capabilities for distinguishing naturally-occurring pathogens from intentional.


One Health surveillance of humans and potential sources of future spillover or ongoing exposure for this organism and future pathogens, including both evolutionary hosts (e.g., bats) and transmission hosts (e.g., heavily trafficked and farmed wildlife and domestic food and companion species), inclusive of environmental, demographic, and occupational risk factors.


### What has been published about medical care?

In [101]:
relevant_articles(task8, 5, only_covid19=True)

Resources to support skilled nursing facilities and long term care facilities.


Mobilization of surge medical staff to address shortages in overwhelmed communities


Age-adjusted mortality data for Acute Respiratory Distress Syndrome (ARDS) with/without other organ failure – particularly for viral etiologies


Extracorporeal membrane oxygenation (ECMO) outcomes data of COVID-19 patients


Outcomes data for COVID-19 after mechanical ventilation adjusted for age.


...

Use of AI in real-time health care delivery to evaluate interventions, risk factors, and outcomes in a way that could not be done manually.


Best practices and critical challenges and innovative solutions and technologies in hospital flow and organization, workforce protection, workforce allocation, community-based support resources, payment, and supply chain management to enhance capacity, efficiency, and outcomes.


Efforts to define the natural history of disease to inform clinical care, public health interventions, infection prevention control, transmission, and clinical trials


Efforts to develop a core clinical outcome set to maximize usability of data across a range of trials


Efforts to determine adjunctive and supportive interventions that can improve the clinical outcomes of infected patients (e.g. steroids, high flow oxygen)


### What has been published about information sharing and inter-sectoral collaboration?

In [102]:
relevant_articles(task9, 5, only_covid19=True)

Methods for coordinating data-gathering with standardized nomenclature.


Sharing response information among planners, providers, and others.


Understanding and mitigating barriers to information-sharing.


How to recruit, support, and coordinate local (non-Federal) expertise and capacity relevant to public health emergency response (public, private, commercial and non-profit, including academic).


Integration of federal/state/local public health surveillance systems.


...

Action plan to mitigate gaps and problems of inequity in the Nation’s public health capability, capacity, and funding to ensure all citizens in need are supported and can access information, surveillance, and treatment.


Measures to reach marginalized and disadvantaged populations.


Data systems and research priorities and agendas incorporate attention to the needs and circumstances of disadvantaged populations and underrepresented minorities.


Mitigating threats to incarcerated people from COVID-19, assuring access to information, prevention, diagnosis, and treatment.


Understanding coverage policies (barriers and opportunities) related to testing, treatment, and care


## Widget: Pick a Task

In [103]:
#########################################################################################################################
#
#    Title: Topic Modeling: Finding Related Articles
#    Author: Daniel Wolffram
#    Date: 2020
#    Code version: 10
#    Availability: https://www.kaggle.com/danielwolffram/topic-modeling-finding-related-articles?scriptVersionId=30463507
# 
#########################################################################################################################
def relevant_articles_for_task():
    tasks={'What is known about transmission, incubation, and environmental stability?': task1,
           'What do we know about COVID-19 risk factors?': task2, 
           'What do we know about virus genetics, origin, and evolution?': task3, 
           'What do we know about non-pharmaceutical interventions?': task4,
           'What do we know about vaccines and therapeutics?': task5, 
           'What has been published about ethical and social science considerations?': task6, 
           'What do we know about diagnostics and surveillance?': task7,
           'What has been published about medical care?': task8, 
           'What has been published about information sharing and inter-sectoral collaboration?': task9}

    def main_function(bullet, task, k=5, year_range=[1950, 2020], only_covid19=False):
        relevant_articles([bullet], k, lower=year_range[0], upper=year_range[1], only_covid19=only_covid19)
        bulletW.options = tasks[task]    

    yearW = widgets.IntRangeSlider(min=1950, max=2020, value=[2010, 2020], description='Year Range', 
                                   continuous_update=False, layout=Layout(width='40%'))
    covidW = widgets.Checkbox(value=True,description='Only COVID-19-Papers',disabled=False, indent=False, layout=Layout(width='20%'))
    kWidget = widgets.IntSlider(value=10, description='k', max=50, min=1, layout=Layout(width='30%'))

    taskW = widgets.Dropdown(options=tasks.keys(), layout=Layout(width='90%', height='50px'), description='Task:')
    init = taskW.value
    bulletW = widgets.Select(options=tasks[init], layout=Layout(width='90%', height='200px'), description='Bullet Point:')

    widget = widgets.interactive(main_function, task=taskW, bullet=bulletW, k=kWidget, year_range=yearW, only_covid19=covidW)
    
    controls = VBox([HBox([widget.children[2], widget.children[3], widget.children[4]], layout=Layout(width='90%', justify_content='space-around')),
                     widget.children[1],
                     widget.children[0]], layout=Layout(align_items='center'))
    
    output = widget.children[-1]
    display(VBox([controls, output]))

In [104]:
relevant_articles_for_task()

## Widget: Free Text Search

In [105]:
#########################################################################################################################
#
#    Title: Topic Modeling: Finding Related Articles
#    Author: Daniel Wolffram
#    Date: 2020
#    Code version: 10
#    Availability: https://www.kaggle.com/danielwolffram/topic-modeling-finding-related-articles?scriptVersionId=30463507
# 
#########################################################################################################################
def relevant_articles_for_text():    
    textW = widgets.Textarea(
        value='',
        placeholder='Type something',
        description='',
        disabled=False,
        layout=Layout(width='90%', height='200px')
    )

    yearW = widgets.IntRangeSlider(min=1950, max=2020, value=[2010, 2020], description='Year Range', 
                               continuous_update=False, layout=Layout(width='40%'))
    covidW = widgets.Checkbox(value=True,description='Only COVID-19-Papers',disabled=False, indent=False, layout=Layout(width='25%'))
    kWidget = widgets.IntSlider(value=10, description='k', max=50, min=1, layout=Layout(width='25%'))

    button = widgets.Button(description="Search")

    display(VBox([HBox([kWidget, yearW, covidW], layout=Layout(width='90%', justify_content='space-around')),
        textW, button], layout=Layout(align_items='center')))

    def on_button_clicked(b):
        clear_output()
        display(VBox([HBox([kWidget, yearW, covidW], layout=Layout(width='90%', justify_content='space-around')),
            textW, button], layout=Layout(align_items='center')))        
        relevant_articles(textW.value, kWidget.value, yearW.value[0], yearW.value[1], covidW.value)

    button.on_click(on_button_clicked)

In [106]:
relevant_articles_for_text()

covid 19 covid-19 corona virus


## Finding Related Sentences (GloVe and cosine_similarity)

In [107]:
from scipy.spatial import distance
from scipy import spatial

### Load Data "Kaggle_Covid19_All_Sources.csv"

In [108]:
all_data = pd.read_csv('Kaggle_Covid19_All_Sources.csv')

In [109]:
all_data['publish_year'] = all_data.publish_time.str[:4].fillna(-1).astype(int)

In [110]:
tf = joblib.load('tf.csv')

In [111]:
lda_tf = joblib.load('lda.pkl')

In [112]:
topic_dist = pd.read_csv('topic_dist.csv')

### Split Text into Sentences

In [113]:
sentences = []
for s in all_data.text_body:
    sentences.append(sent_tokenize(s))
sentences = [y for x in sentences for y in x]

### Extract word vectors

Download GloVe: Global Vectors for Word Representation Wikipedia 2014 + Gigaword 5
Ref: Jeffrey Pennington, Richard Socher, and Christopher D. Manning. 2014. GloVe: Global Vectors for Word Representation. 


In [66]:
!unzip glove*.zip

Archive:  glove.6B.zip
  inflating: glove.6B.50d.txt        
  inflating: glove.6B.100d.txt       
  inflating: glove.6B.200d.txt       
  inflating: glove.6B.300d.txt       


In [114]:
#########################################################################################################################
#
#    Title: An Introduction to Text Summarization using the TextRank Algorithm (with Python implementation)
#    Author: Prateek Joshi
#    Date: 2018
#    Code version: 1
#    Availability: https://www.analyticsvidhya.com/blog/2018/11/introduction-text-summarization-textrank-python/
#########################################################################################################################

# Extract word vectors
word_embeddings = {}
f = open('glove.6B.100d.txt', encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    word_embeddings[word] = coefs
f.close()

### Clean sentences

In [115]:
def fun_clean_sentences(sentences):

    # remove punctuations, numbers and special characters
    clean_sentences = pd.Series(sentences).str.replace("[^a-zA-Z]", " ")

    # make alphabets lowercase
    clean_sentences = [s.lower() for s in clean_sentences]
    stop_words = stopwords.words('english')
    stop_words.extend(['background', 'methods', 'introduction', 'conclusions', 'results', 
                   'purpose', 'materials', 'discussions','methodology','result analysis'])
    def remove_stopwords(sen):
        sen_new = " ".join([i for i in sen if i not in stop_words])
        return sen_new
    clean_sentences = [remove_stopwords(r.split()) for r in clean_sentences]
    return clean_sentences
    

In [116]:
clean_sentences = fun_clean_sentences(sentences)

### Vector Representation of Text-body Sentences

In [117]:
#########################################################################################################################
#
#    Title: An Introduction to Text Summarization using the TextRank Algorithm (with Python implementation)
#    Author: Prateek Joshi
#    Date: 2018
#    Code version: 1
#    Availability: https://www.analyticsvidhya.com/blog/2018/11/introduction-text-summarization-textrank-python/
#########################################################################################################################
        
sentence_vectors = []
for i in clean_sentences:
    if len(i) != 0:
        v = sum([word_embeddings.get(w, np.zeros((100,))) for w in i.split()])/(len(i.split())+0.001)
    else:
        v = np.zeros((100,))
    sentence_vectors.append(v)

### Vector Representation of Question Sentences

In [118]:
questions = ["Range of incubation periods for the disease in humans (and how this varies across age and health status) and how long individuals are contagious, even after recovery. days, years, time, period, childern, kid, young, senior, adults, old, COVID-19, 2019-nCov, 'coronavirus', 'cov-2', 'sars-cov-2', 'sars-cov, hcov, 2019-ncov",
"Prevalence of asymptomatic shedding and transmission (e.g., particularly children).",
"Seasonality of transmission. spring, winter, summer, autumn, fall, cold, hot, warm",
"Physical science of the coronavirus (e.g., charge distribution, adhesion to hydrophilic/phobic surfaces, environmental survival to inform decontamination efforts for affected areas and provide information about viral shedding).",
"Persistence and stability on a multitude of substrates and sources (e.g., nasal discharge, sputum, urine, fecal matter, blood).",
"Persistence of virus on surfaces of different materials (e,g., copper, stainless steel, plastic,paper, wood, metal, food, cloth, hair, skin, porcelain).",
"Natural history of the virus and shedding of it from an infected person",
"Implementation of diagnostics and products to improve clinical processes",
"Disease models, including animal models for infection, disease and transmission",
"Tools and studies to monitor phenotypic change and potential adaptation of the virus",
"Immune response and immunity",
"Effectiveness of movement control strategies to prevent secondary transmission in health care and community settings",
"Effectiveness of personal protective equipment (PPE) and its usefulness to reduce risk of transmission in health care and community settings, mask, google, gloves",
"Role of the environment in transmission"]

In [119]:
#########################################################################################################################
#
#    Title: An Introduction to Text Summarization using the TextRank Algorithm (with Python implementation)
#    Author: Prateek Joshi
#    Date: 2018
#    Code version: 1
#    Availability: https://www.analyticsvidhya.com/blog/2018/11/introduction-text-summarization-textrank-python/
#########################################################################################################################

clean_questions = fun_clean_sentences(questions)
question_vectors = []
for i in clean_questions:
    if len(i) != 0:
        v = sum([word_embeddings.get(w, np.zeros((100,))) for w in i.split()])/(len(i.split())+0.001)
    else:
        v = np.zeros((100,))
    question_vectors.append(v)

### Cosine Similarity Matrix

In [120]:
sim_mat= cosine_similarity(sentence_vectors, question_vectors)
sim_mat_df = pd.DataFrame(sim_mat)

In [121]:
question_vectors[:10]

[array([ 0.02598357,  0.28678034,  0.14133468, -0.05999389, -0.25050776,
         0.06722982,  0.0099764 , -0.0545159 , -0.12801499, -0.107351  ,
        -0.12949992,  0.04054813,  0.39085074, -0.07860075,  0.38762581,
        -0.13924788,  0.03399591, -0.06326686, -0.22715112, -0.0834815 ,
         0.08473672,  0.02601746, -0.09232635,  0.03516977,  0.08181691,
         0.09949684,  0.15261544, -0.28712908, -0.26239698,  0.07720456,
        -0.02225858,  0.2132642 ,  0.1177438 , -0.14531829,  0.01536673,
         0.01753675, -0.08392329,  0.10756533, -0.15314619, -0.0931374 ,
        -0.35264745, -0.08428739, -0.03105246, -0.1670065 ,  0.24759063,
         0.0225693 ,  0.32225427, -0.2098243 , -0.01991602, -0.32972661,
         0.1620186 , -0.32015048, -0.06429728,  0.63006004,  0.06188462,
        -1.13237943, -0.04947465, -0.13596332,  0.90605182,  0.39211474,
        -0.04900051,  0.70174528,  0.00291355, -0.07888153,  0.39033658,
         0.38007339,  0.03124955, -0.19778275,  0.3

Print out the top n most similar sentences with given question sentences (starting from 1)

In [122]:
def top_n_similar_sentences_glove_cosine(n_question, n=10):
    print(f"QUESTION #{n_question}: {questions[n_question-1]}\n\n")
    top_n_similar_sentences = []
    top_n_sim = sim_mat_df.loc[:,(n_question-1)].nlargest(n)
    for idx in top_n_sim.index:
        top_n_similar_sentences.append(sentences[idx])
    return(top_n_similar_sentences)

## Glove-Cosine after LDA selected articles

### Get "Nearest" Papers (in Topic Space)

In [123]:
#########################################################################################################################
#
#    Title: Topic Modeling: Finding Related Articles
#    Author: Daniel Wolffram
#    Date: 2020
#    Code version: 10
#    Availability: https://www.kaggle.com/danielwolffram/topic-modeling-finding-related-articles?scriptVersionId=30463507
# 
#########################################################################################################################
def get_k_nearest_docs(doc_dist, k=5, lower=1950, upper=2020, only_covid19=False):
    '''
    doc_dist: topic distribution (sums to 1) of one article
    
    Returns the index of the k nearest articles (as by Jensen–Shannon divergence in topic space). 
    '''
    
    relevant_time = all_data.publish_year.between(lower, upper)
    
    if only_covid19:
        is_covid19_article = all_data.text_body.str.contains('COVID-19|SARS-CoV-2|2019-nCov')
        topic_dist_temp = topic_dist[relevant_time & is_covid19_article]
        
    else:
        topic_dist_temp = topic_dist[relevant_time]
         
    distances = topic_dist_temp.apply(lambda x: jensenshannon(x, doc_dist), axis=1)
    k_nearest = distances[distances != 0].nsmallest(n=k).index
        
    return k_nearest

In [124]:
#########################################################################################################################
#
#    Title: Topic Modeling: Finding Related Articles
#    Author: Daniel Wolffram
#    Date: 2020
#    Code version: 10
#    Availability: https://www.kaggle.com/danielwolffram/topic-modeling-finding-related-articles?scriptVersionId=30463507
# 
#########################################################################################################################
def relevant_articles_doc_id(tasks, k=3, lower=1950, upper=2020, only_covid19=False):
    tasks = [tasks] if type(tasks) is str else tasks 
    
    tasks_tf = tf_vectorizer.transform(tasks)
    tasks_topic_dist = pd.DataFrame(lda_tf.transform(tasks_tf))

    for index, bullet in enumerate(tasks):

        recommended = get_k_nearest_docs(tasks_topic_dist.iloc[index], k, lower, upper, only_covid19)
        recommended = pd.DataFrame(all_data.iloc[recommended])
        recommended_doc_id = recommended.doc_id
    return recommended_doc_id

In [125]:
stop_words = stopwords.words('english')
stop_words.extend(['background', 'methods', 'introduction', 'conclusions', 'results', 
               'purpose', 'materials', 'discussions','methodology','result analysis'])

def remove_stopwords(sen):
    sen_new = " ".join([i for i in sen if i not in stop_words])
    return sen_new

def fun_clean_sentences(sentences):

    clean_sentences = pd.Series(sentences).str.replace("[^a-zA-Z]", " ")

    clean_sentences = [s.lower() for s in clean_sentences]

    clean_sentences = [remove_stopwords(r.split()) for r in clean_sentences]
    return clean_sentences
    

In [126]:
def top_n_similar_sentences(n_question, sentence_df, n=10):
    print(f"QUESTION #{n_question}: {task1[n_question-1]}\n\n")
    top_n_similar_sentences = []
    for i in range(len(sentence_df)):
        sentence_df['sim_score'][i] = sentence_df['sim_score'][i][0][0]
    top_n_similar_sentences = sentence_df.sort_values(by=['sim_score'], ascending=False).iloc[0:n,]
    return(top_n_similar_sentences)

In [127]:
#########################################################################################################################
#
#    Title: An Introduction to Text Summarization using the TextRank Algorithm (with Python implementation)
#    Author: Prateek Joshi
#    Date: 2018
#    Code version: 1
#    Availability: https://www.analyticsvidhya.com/blog/2018/11/introduction-text-summarization-textrank-python/
#########################################################################################################################

def glove_vectorizer(clean_sentences):
    sentence_vectors = []
    for i in clean_sentences:
        if len(i) != 0:
            v = sum([word_embeddings.get(w, np.zeros((100,))) for w in i.split()])/(len(i.split())+0.001)
        else:
            v = np.zeros((100,))
        sentence_vectors.append(v)
    return sentence_vectors

In [128]:
def get_recommanded_articles(question_no, top_n_articles, lower=1950, upper=2020, only_covid19=True):
    recommanded_articles = pd.DataFrame(columns=all_data.columns)
    recommended_doc_id = relevant_articles_doc_id(tasks=task1[question_no-1], k=top_n_articles, lower=lower, upper=upper, only_covid19=only_covid19)    
    recommended_doc_id = list(recommended_doc_id)
    for i in range(len(recommended_doc_id)):
        recommanded_articles = recommanded_articles.append(all_data[all_data.doc_id == recommended_doc_id[i]])[['doc_id', 'title', 'text_body', 'doi']]
    return recommanded_articles

In [129]:
def sent_tokenizer(text):
    sentences = []
    for s in text:
        sentences.append(sent_tokenize(s))
    sentences = [y for x in sentences for y in x]
    return sentences

In [130]:
def top_n_similar_sentences_LDA_glove_cosine(question_no, top_n_articles, top_n_sentences, only_covid19=True):
    recommanded_articles = get_recommanded_articles(question_no, top_n_articles, only_covid19=only_covid19)
    sentence_df = pd.DataFrame(columns = ['doc_id','title','sentences','clean_sentences', 'sentence_vectors', 'sim_score'])
    for i in range(len(recommanded_articles)):
        article_df = pd.DataFrame(columns = ['doc_id','title','sentences'])    
        sentences = sent_tokenize(recommanded_articles.iloc[i].text_body)
        article_df['sentences'] = sentences
        article_df['doc_id'] = recommanded_articles.iloc[i].doc_id
        article_df['title'] = recommanded_articles.iloc[i].title
        article_df['doi'] = recommanded_articles.iloc[i].doi
        sentence_df = sentence_df.append(article_df, ignore_index=True)
    clean_questions = fun_clean_sentences(questions[question_no-1])
    question_vectors = glove_vectorizer(clean_questions)
    for i in range(len(sentence_df)):
        sentence_df.loc[i,'clean_sentences'] = fun_clean_sentences(sentence_df.iloc[i].sentences)
        sentence_df.loc[i,'sentence_vectors'] = glove_vectorizer(sentence_df.iloc[i].clean_sentences)
        sentence_df.loc[i,'sim_score'] = cosine_similarity(sentence_df.iloc[i].sentence_vectors, question_vectors)
    similar_sentences = top_n_similar_sentences(question_no, sentence_df, top_n_sentences)
    for i in range(len(similar_sentences)):
        l,n = similar_sentences.iloc[i][['doi','title']].values
        if n !=n:
            n = 'no title'
            h = '<a href="' + l + '" target="_blank">'+ n + '</a>'
        else:
            h = '<a href="' + l + '" target="_blank">'+ n + '</a>'
        print(similar_sentences.iloc[i].sentences)
        display(HTML(h))

## Comparations of results of two approches

Question 1: 

In [131]:
top_n_similar_sentences_glove_cosine(1,30)

QUESTION #1: Range of incubation periods for the disease in humans (and how this varies across age and health status) and how long individuals are contagious, even after recovery. days, years, time, period, childern, kid, young, senior, adults, old, COVID-19, 2019-nCov, 'coronavirus', 'cov-2', 'sars-cov-2', 'sars-cov, hcov, 2019-ncov




['Depending on how much time has passed since exposure to the primary infected individual, those infected may not yet be symptomatic -this period of time between infection and symptoms is an important epidemiological trait of an infectious disease called the incubation period.',
 'It is known that people infected with hepatitis A virus experience an incubation period of 28 days ranging 15-50 days to become ill [16, 24] .',
 'In microbiological terms, the book describes the emergence and spread of a pandemic whose infection and mortality rates are 100%, with an incubation period of a few days, whose symptoms make those infected extremely dangerous to society, and for which there is no treatment.',
 'Of the 154 SARS patients who satisfied the WHO definition for SARS, their age range was 20-80 years (mean 41.5 years).',
 'If we assume a particular infectious pathogen has a contact rate, = 0.75, with an incubation period of two days, = 0.5, and an infectious period of five days, = 0.2 (sim

In [162]:
top_n_similar_sentences_LDA_glove_cosine(1, 5, 30)

QUESTION #1: Range of incubation periods for the disease in humans (and how this varies across age and health status) and how long individuals are contagious, even after recovery.


A detailed analysis of one of the early COVID-19 clusters by Chan and colleagues 19 revealed symptomatic infections in five adult members of the same household, while a child in the same household aged 10 years was infected but remained asymptomatic, potentially indicating biological differences in the risk of clinical disease driven by age.


In any case, if the age distribution of cases reported here was to be confirmed and the epidemic were to progress globally, we would expect an increase in respiratory mortality concentrated among people aged 30 years and older.


Adjustment for the age demographics of China confirmed a deficit of infections among children, with a RR below 0·5 in patients younger than 15 years (figure 1).


If the current intervention continues, the number of infected individuals is expected to peak in early March 2020 (80 days since initiation) with a peak population size of 827 (421-1232) infectious individuals in China.


Even in the period after Jan 18, 2020, when awareness of the outbreak increased, a shorter delay between symptom onset and seeking care at a hospital or clinic was seen for international patients than for those in mainland China (Wilcoxon test p<0·0001).


...

[13] [14] [15] [16] The incubation period is a useful parameter to guide isolation and contact tracing; based on existing data, the disease status of a contact should be known with near certainty after a period of observation of 14 days.


We calculated the effective reproduction number (R eff ) of the simulation as the average number of secondary cases produced by each infected person in the presence of isolation and contact tracing.


As an example (figure 1), a person infected with the virus could potentially produce three secondary infections (because three is drawn from the negative binomial distribution), but only two transmissions might occur before the case is isolated.


Facing the rapidly rising epidemic, the Chinese government has timely amended the Law of the PRC on the Prevention and Treatment of Infectious Diseases on 20 th January 2020 to include the 2019-nCov as a class-B infection but manage it as a class-A infection due to its severity [8] .


This occurs because with better contact tracing it becomes possible to control outbreaks with higher numbers of weekly cases.


Question 2: 

In [133]:
top_n_similar_sentences_glove_cosine(2,30)

QUESTION #2: Prevalence of asymptomatic shedding and transmission (e.g., particularly children).




['F I G U R E 1 Viral load variation among the groups of asymptomatic contacts, children possibly not transmitting RSV, and children possibly transmitting RSV (11 days) , and additional studies related to RSV infection among these patients would understand the dynamic of this type of viral infection.',
 'Furthermore, mother-to-child transmission of GBV-C reduces the vertical transmission of HIV-1 from GBV-C/HIV-1 coinfected mothers [21] , and recently it was reported that accidental GBV-C acquisition via transfusion is associated with a significant reduction in mortality in HIV-infected individuals [22] .',
 'Both pregnancies that end with vertical transmission of the infection, the mothers were in third stage of HIV-infection with severe immune suppression (CD4 <200/μl) and high viral load of HIV > 100 000 c/μl.',
 'Infants carrying H4 or H6 haplotypes had increased risk of IU HIV-1-infection, whereas H2 haplotype carriers were less likely to be infected during pregnancy compared to i

In [163]:
top_n_similar_sentences_LDA_glove_cosine(2, 5, 30)

QUESTION #2: Prevalence of asymptomatic shedding and transmission (e.g., particularly children).


After the stage, likely after mid-January, 2020, the virus further spread to the family via infected adults to cause intrafamilial transmission, especially transmission to the elderly and children, who are vulnerable to the infection.


With more diagnostic detection done, the proportion of mild infections mainly in children and young adults became higher.


1B , during the emerging stage of the SARS-CoV-2 outbreak, the infection was disseminated by person-to-person transmission in the community almost exclusively among adults.


First, asymptomatic cases were diagnosed based on positive viral nucleic acid test results, but without any COVID-19 symptoms, such as fever, gastrointestinal, or respiratory symptoms, and no significant abnormalities on chest radiograph 7, 8 However, the transmission of COVID-19 through asymptomatic carriers via person-to-person contact was observed in many reports.


9 Similarly, at the beginning of the 2009 pandemic H1N1 influenza outbreak, the percentage age distributions for mortality and morbidity for patients with severe pneumonia show a marked shift to persons between the ages of 5 and 59 years, as compared with distributions observed during previous periods of epidemic influenza.


...

However, this study involved a population of only 18 patients, including one asymptomatic patient.


12 Other suggested reasons include children having a more active innate immune response, healthier respiratory tracts because they have not been exposed to as much cigarette smoke and air pollution as adults, and fewer underlying disorders.


Nevertheless, those aged 15e24 and 25e44 years experienced sharply elevated death rates.


14 Most, if not all, of the infants received regular immunizations, including BCG, in China and other Asian countries, and it is well known that influenza can cause more ARDS in the adults, yet very less in children.


Usually they recover within 1e2 weeks after the onset of the disease.


Question 3: 

In [135]:
top_n_similar_sentences_glove_cosine(3,30)

QUESTION #3: Seasonality of transmission. spring, winter, summer, autumn, fall, cold, hot, warm




['The weather during spring and autumn is usually warm during the days and cooler at nights.',
 'The spring and autumn are relatively short when compared with the duration of summer and winter.',
 'Adenovirus was most likely to be detected in summer, followed by spring, winter and autumn.',
 'RSV was most likely to be detected in winter, followed by spring, autumn, and summer.',
 'IFVB was active in cold winter and IFVA was active in autumn, while both were rarely detected in summer.',
 'positivity was low in spring-summer and high in fall-winter, although no significant differences were observed (38.8, 40.9, 47.3 and 49.5% in spring, summer, fall and winter, respectively).',
 'The area has a continental monsoon climate, winter is longer than other seasons, and summer is warm.',
 'In detail, ADV infection was prevalent in summer (20/34, 58.8%) and spring (10/34, 29.4%) seasons, FLU was prevalent in spring (7/14, 50%) and winter (4/14, 28.5%), HMPV in winter (2/4, 50%), HCoV in autumn/w

In [164]:
top_n_similar_sentences_LDA_glove_cosine(3, 5, 30)

QUESTION #3: Seasonality of transmission.


In 2003, the spring festival transport period started from Jan 17 to Feb 25, 2020 and coincided with the peak incidence [ Figure 1F , purple box].


Both outbreaks happened in the winter, when the two provinces have similar climate patterns suitable for virus survival and spread.


Coincidentally, the SARS outbreak duration also coincided with the Chinese spring festival.


The spring festival travel period in 2020 started from Jan 10 to Feb 18, which coincided with the rapid increase in SARS-CoV-2 cases between Jan 10 and 22, 2020 [ Figure 1F , red box].


Similar to the SARS outbreak, this outbreak also occurred during the spring festival, the most important of the Chinese traditional festivals, when 3 billion people travel throughout the country [6] .


...

13, 14 A narrow window of exposure could be defined for a subset of patients who had a short stay in Wuhan, at a time when the epidemic was still localised to Wuhan.


Although the data so far suggest that the disease is mild in most cases and that the case fatality rate is currently reported to be lower than SARS or MERS, the situation is likely to go on for months and could cause severe disruption in countries that are not well prepared.


A series of epidemiological criteria were required for COVID-19 testing, including travel history to Wuhan within the past 2 weeks; residence in Wuhan within the past 2 weeks; contact with individuals from Wuhan (with fever and respiratory symptoms) within the past 2 weeks; and being part of an established disease cluster.


Setting the upper limit of cumulative incidence (K) to 50,000, 60,000, or 70,000, the end date of incidences will be in 56 days (Mar 6, 2020), 60 days (Mar 10, 2020), or 62 days (Mar 12, 2020), respectively.


As of Jan 31, 2020, province-level epidemic curves are only available by date of reporting, rather than date of symptom onset, which usually inflates recent case counts if detection has increased.


Question 4: 

In [137]:
top_n_similar_sentences_glove_cosine(4,30)

QUESTION #4: Physical science of the coronavirus (e.g., charge distribution, adhesion to hydrophilic/phobic surfaces, environmental survival to inform decontamination efforts for affected areas and provide information about viral shedding).




['This study, aimed at controlling the microbial bioburden and infectious reservoir on inanimate, everyday objects, builds upon our previous work in the development of photo-activated, antimicrobial polymers for the prevention of medical device associated infection [34, 35, 37] , thereby expanding scope of photoactive materials for potential infection control applications in a healthcare setting.',
 'Understanding infectious disease transmission is a complex problem due to the many factors involved, such as environmental conditions, human activity (e.g., more crowding in buildings during wintertime), hygiene, and host susceptibility.',
 'The review addresses the following topics: a) the burden of waterborne disease, both globally and in the United States, with discussion of the dramatic underreporting of disease; b) etiology ofwaterborne disease, e.g., direct contamination of water by enteric pathogens and growth of opportunistic pathogens within distribution system biofilms; c) charac

In [165]:
top_n_similar_sentences_LDA_glove_cosine(4, 5, 30)

QUESTION #4: Physical science of the coronavirus (e.g., charge distribution, adhesion to hydrophilic/phobic surfaces, environmental survival to inform decontamination efforts for affected areas and provide information about viral shedding).


[8] supports that data from various technological products can help enrich health databases, provide more accurate, efficient, comprehensive and real-time information on outbreaks and their dispersal, thus aiding in the provision of better urban fabric risk management decisions.


4 I anticipate international efforts in these areas over the coming decade will enable the tapping of useful new biological functions and processes, methods for controlling infection, and the deployment of symbiotic or subclinical viruses in new therapies and biotechnologies that are so crucially needed.


This is valid as smart cities host a rich array of technological products [6, 7] that can assist in early detection of outbreaks; either through thermal cameras or Internet of Things (IoT) sensors, and early discussions could render efforts towards better management of similar situations in case of future potential outbreaks, and to improve the health fabric of cities generally.


2 These frame Australia's Medical Research and Innovation Priorities, which include antimicrobial resistance, global health and health security, drug repurposing and translational research infrastructure, 15 capturing many of the key elements of this CTI Special Feature.


Multidisciplinary research in biomedical, social, and environmental sciences is required to achieve a deeper understanding of disease transmission and develop more effective systems for emergency response.


...

Beyond the aspect of pandemic preparedness and response, the case of COVID-19 virus and its spread provide a fascinating case study for the thematics of urban health.


Such could lead to an even earlier detection scenario of future virus outbreaks, and in the better curative management of the same, without minimal compromise on urban functions and on an urban economy.


Altogether, there exist significant differences in transmission, pathogenesis, clinical treatment, and vaccine development between these two viruses.


However, while the potential for the data market is understood, such issues like privacy of information, data protection and sharing, and obligatory requirements of healthcare management and monitoring, among others, are critical.


With a collaborated data sharing protocol, it would be possible to have a larger dataset resulting in increased processing capabilities especially with technologies that are powered by artificial intelligence (AI) tools.


Question 5: 

In [139]:
top_n_similar_sentences_glove_cosine(5,30)

QUESTION #5: Persistence and stability on a multitude of substrates and sources (e.g., nasal discharge, sputum, urine, fecal matter, blood).




['A liquid extract of each fecal sample (1:5 dilution) was used for NMH analysis and fecal NMH concentrations were back-calculated for the wet weight of the fecal samples and expressed in ng/g feces.',
 'The presence of surrounding organic material (e.g., blood, saliva, mucus, etc.)',
 'Viral RNAs could be found in nasal discharge, sputum, and sometimes blood or feces.',
 'Viral RNAs could be found in nasal discharge, sputum, and sometimes blood or feces.',
 'In particular, the concentration of PEDV nucleic acid in individual pig rectal swabs was significantly lower than oral fluid or penbased fecal samples.',
 'Shedding in urine was not strictly associated with high blood loads (no significant correlation between blood and urine loads was observed, p S > 0.05) or blood contamination of the urine sample (data not shown).',
 'feces, blood or tissues) and viral detection method (e.g.',
 'A positive result from blood or pleural fluid (e.g., for meningitis, cerebrospinal fluid) samples by 

In [161]:
top_n_similar_sentences_LDA_glove_cosine(5, 5, 30)

QUESTION #5: Persistence and stability on a multitude of substrates and sources (e.g., nasal discharge, sputum, urine, fecal matter, blood).


In these traditional methods, the samples need pre-lysis in an appropriate buffer to release nucleic acids from viral particles before binding to the column membrane and multiple centrifugation steps are required to enable binding, washing and elution of extracted nucleic acids.


Among other goals for this work, we plan to evaluate: (1) sensitivity of the SARS-CoV-2 assay against clinical isolates and patient samples-including sputum, throat, and nasal swabs-some of which may be challenging sample types to test; (2) specificity at both the species and subspecies levels against highly related viruses.


By following a simple lysis/binding-washing-elution protocol shown in Scheme2, 10 5 copies of SARS-CoV-2 pseudovirus in 200 μL serum samples were extracted and subject to RT-PCR analysis.


Three types of aerosol samples were collected: 1) Aerosol samples of total suspended particles (TSP) with no upper size limit to quantify RNA concentration of SARS-CoV-2 aerosol; 2) Aerodynamic size segregated aerosol samples to determine the size distribution of airborne SARS-CoV-2; 3) Aerosol deposition samples to determine the deposition rate of airborne SARS-CoV-2.


This result suggests that our pcMNPs-based viral RNA extraction protocol not only exhibits nearly 100% RNA extraction efficiency in serum samples, but also provides high-purity products without PCR inhibitors.


...

While the mass to charge (m/z) ratio of a peptide or protein (MS1) may be a specific diagnostic in some materials, the majority of LCMS methods employ tandem MS in which the peptide or protein parent ion is subjected to gas phase collision to produce fragment ions.


A very low frequency of the residues will create very large peptides, or undigested ("intact") proteins in some cases, that are difficult to detect and fragment.


Because no centrifugation steps are required, MNPs-based methods allow fully automated nucleic acid purification, which is highly important in current SARS-CoV-2 diagnosis.


Secondly, pcMNPs have excellent viral RNA binding performances, which results in 10-copy sensitivity and the high linearity over 5 logs of gradient in SARS-CoV-2 viral RNA detection using RT-PCR.


In DDA, the masses of all ions are observed in a relatively wide m/z range (MS1); the MS1 peptide ions meeting user-defined thresholds are subjected to fragmentation.


Question 6: 

In [141]:
top_n_similar_sentences_glove_cosine(6,30)

QUESTION #6: Persistence of virus on surfaces of different materials (e,g., copper, stainless steel, plastic,paper, wood, metal, food, cloth, hair, skin, porcelain).




['Non-animal food-contact swabs were analyzed by surface type (metal garage, metal tabletop, concrete floor, and rubber boot bottoms worn during the experiment).',
 'It can survive on plastic surfaces, stainless steel, glass slides, and paper files.',
 'Thus, the material itself (vinyl chloride, aluminum, plastic, stainless steel) had no effect on the diminishing bacterial numbers under dry conditions.',
 'Virus recovery from surfaces of Styrofoam, nitrile gloves, aluminum foil, Tyvek ® coverall, metal, rubber, plastic, cardboard, and cloth showed no significant differences between the materials at RT, suggesting that storage temperature had a substantial influence on virus survival.',
 'There was a significant difference in appearance between purified HuCoV-229E exposed to stainless steel and that exposed to copper surfaces (Fig.',
 'Virus that had been exposed to copper and brass surfaces demon- was applied to 1-cm 2 coupons of a range of brasses (A and B [early time points only]), c

In [166]:
top_n_similar_sentences_LDA_glove_cosine(6, 5, 30)

QUESTION #6: Persistence of virus on surfaces of different materials (e,g., copper, stainless steel, plastic).


RBD is shown in a space-filled model with colored surface.


Indicated proteins were coated on 96 well plates using CBS buffer over night at 4℃.


[13] [14] In the control and diagnosis towards SARS-CoV-2 currently, silica-based spin column RNA extraction methods are widely used, in which a silica membrane or glass fiber is applied to bind nucleic acids.


Bare magnetic nanoparticles (MNPs) were prepared based on a simple coprecipitation protocol as previously reported.


ACE2 is shown as gray tube model.


...

For the disinfection of small surfaces ethanol (62%e71%; carrier tests) revealed a similar efficacy against coronavirus [7] .


Then 100μL/well of the antibody-PSV mixture was added onto the 293T/ACE2 cell wells and incubated for author/funder.


The human embryonic kidney 293T cell line (Cat:CRL-11268) used for pseudovirus (PSV) packaging were purchased from ATCC.


[11] [12] Low extraction quality, on the other hand, may contain a variety of PCR inhibitors, which gives unreliable readouts during amplification.


On the other hand, the interaction patterns in both complex structures' interfaces are somewhat different ( Figure 1A and Table S2 ).


Question 7: 

In [143]:
top_n_similar_sentences_glove_cosine(7,30)

QUESTION #7: Natural history of the virus and shedding of it from an infected person




['When that virus infects people, a new and unknown form of influenza that is transmitted from person to person develops.',
 'When that virus infects people, a new and unknown form of influenza that is transmitted from person to person develops.',
 '27 It is important\n During the course of their life, humans are infected by several strains of influenza virus.',
 "Scientists think that the occasional outbreaks of the disease occur because the virus ''jumps'' from an infected animal to a person (a rare event) and then is transmitted between people by direct contact with infected blood or other body fluids or parts.",
 'Many of these respiratory viruses spread easily from person to person, or can be spread from an animal reservoir (23) .',
 'This is probably because, compared with some other viruses, the flu virus is transmitted from one person to another very quickly and affects many people.',
 'Based on finding that the utilized virus was virulent and caused EBOV hemorrhagic disease, w

In [167]:
top_n_similar_sentences_LDA_glove_cosine(7, 5, 30)

QUESTION #7: Natural history of the virus and shedding of it from an infected person


Before viruses in wildlife make a jump to infect human beings, they usually accumulate a series of mutations in their viral genomes [42] and invade human beings as a result of human occupation of their normal ecosystem, as exemplified with a story of initial human infection by HIV carried by chimpanzees in rainforests of West Africa [43, 44] .


cough, lethargy, myalgia) infected individuals was identified early in the course of this outbreak, with human-to-human transmission detected in international case series [7] .


At the outset, SARS-COVs might have a species barrier before it can be transmitted to humans.


The increased vulnerability of human beings in winter time and the increased human exposure to wild animals during holidays made infection to SARS-COV-2 more likely.


This mini-review evaluated the common epidemiological patterns of both SARS epidemics in China and identified cold, dry winter as a common environmental condition conducive for SARS virus infection to human beings.


...

The isolated virus showed more than 95% genome sequence identity with human and civet SARS-CoVs.


Subsequent case investigations also showed that SARS-CoV had the capability to multiply and continuously undergo human-to-human transmission [ Figure 1D ]; at least four generations of cases were identified from one original patient.


Prior to December 2019 when clusters of pneumonia cases with unknown aetiology were detected in Wuhan, China, only two additional strains of coronaviruses had caused outbreaks of severe acute respiratory disease around the world [3] .


On 31 December 2019, local hospitals in Wuhan, China reported that they had detected a series of cases of Novel Coronavirus-infected pneumonia to the World Health Organization (WHO) [1] .


Although the origins and the occurrences of SARS-CoV-2 are both unclear, the control measures for the current epidemic should focus on immediate cut-off of transmission of the disease and through disinfection of infected locations.


Question 8: 

In [145]:
top_n_similar_sentences_glove_cosine(8,30)

QUESTION #8: Implementation of diagnostics and products to improve clinical processes




['There is a need for availability of data and information on humanvector-pathogen-ecosystem interfaces, drugs and vaccines development as well as diagnostics techniques and tools from preclinical to clinical levels.',
 'Improving the optimization and implementation of protocols suitable for clinical samples will no doubt improve microbial diagnosis in clinical practice.',
 'These techniques are expected to greatly accelerate the identification of specific phage binders, facilitating mAb development for use in research, clinical diagnostics, and pharmaceuticals for the treatment of human disease.',
 'Integration of knowledge about microbiology and immunology, establishment of efficient vaccine development strategies, and streamlining of regulatory approval processes may facilitate this trend.',
 'Tools from information technology and progress in microbiology will reduce diagnostic uncertainty and improve antimicrobial dosing, selection, and treatment duration.',
 'There were 8 key elem

In [168]:
top_n_similar_sentences_LDA_glove_cosine(8, 5, 30)

QUESTION #8: Implementation of diagnostics and products to improve clinical processes


13 Incremental improvements in rapid sample preparation techniques, chromatography, and data processing have also contributed to the increasing use of LCMS-based clinical testing.


Ongoing SARS-CoV-2 sequencing is key to developing and monitoring diagnostics and similar surveillance tools.


The subfractionation of existing heparin preparations against anticoagulant activities (with proven low-toxicity profiles, good bioavailability and industrial-scale manufacturing) for off-label pathologies, provides an attractive strategy for quickly and effectively responding to COVID-19 and for the development of next generation heparin-based therapeutics.


Today, protein array and antibody-based methods are falling out of favor in both research and clinical diagnostics, due in large part to the improvements in LCMS technology.


Although much simpler and faster than spin column-based methods, most MNPs-based extraction strategies still contain multiple processing steps such as lysis, binding, washing and elution, which increases operational difficulties in real clinical diagnosis.


...

Our in silico developed materials facilitate both global and targeted analysis by providing all necessary materials for both DDA and DIA investigation of these materials through the production of FASTA databases, spectral libraries and a list of predicted PTMs investigators should consider when searching with historic peptide search engines.


In vitro validation of this method is required and outside the scope of this paper given our lack of access to such samples.


Therefore, it is important to characterize these other pathogens, for both patient diagnostics and outbreak response.


Therefore, the feasibility of automating viral RNA extraction procedure based on our pcMNPs was subsequently evaluated.


Such a lack of understanding limits effective risk assessment, prevention and control of COVID-19 disease outbreaks.


Question 9: 

In [147]:
top_n_similar_sentences_glove_cosine(9,30)

QUESTION #9: Disease models, including animal models for infection, disease and transmission




['More recently it has been shown that novel combinations of alleles have also resulted in new models for human disease such a spontaneous colitis [67] , and that F1 hybrids of CC mice were used to create an improved mouse model for Ebola virus disease [68] including hemorrhagic signs of disease previously not observed in a small animal model.',
 'The ferret is the most suitable disease model for human influenza infection as it displays very human-like disease [20] [21] [22] .',
 'The MA15 virus will enhance the use of the mouse model for SARS because infection with this virus in mice reproduces many aspects of severe human disease, including morbidity, mortality, and pulmonary pathology.',
 'Ferrets are commonly used as experimental models of infection for a variety of respiratory viruses due to their susceptibility to these viruses and the close resemblance of the pathological features to those found in human infections [11, 12] , including the development of severe respiratory and n

In [169]:
top_n_similar_sentences_LDA_glove_cosine(9, 5, 30)

QUESTION #9: Disease models, including animal models for infection, disease and transmission


The second component, f late , models HIV transmission during the asymptomatic stage and the disease stage (after progression to Acquired Immune Deficiency Syndrome (AIDS)).


Furin, along with other PCSK family members, is widely implicated in immune regulation, cancer and the entry, maturation or release of a broad array of evolutionarily diverse viruses including human papillomavirus (HPV), influenza (IAV), Ebola (EboV), dengue (DenV) and human immunodeficiency virus (HIV).


It has been noted that the two previously known human coronaviruses causing epidemic disease and spread, SARS-CoV and MERS-CoV, had a relatively low rate of spread from an individual infected patient (an index referred to as its basic reproductive number-R。).


Coronaviruses have in the past been known to be the etiologic agents of mild upper respiratory infections in humans, similar to the ubiquitous and relatively benign "common cold"-type upper respiratory illnesses induced by the human rhinoviruses in adults and children.


Observed characteristics of the outbreak led us to believe that the cluster of cases was due to "Disease X" (i.e., an infectious disease of previously unknown viral etiology).


...

Although virological investigation is the gold standard for pathogen identification, and the virus has now been confirmed to be a novel coronavirus that is a relative of SARS, laboratory-based outcomes can only be obtained after successfully sequencing the novel virus, which can sometimes be a lengthy process.


SARS-CoV-2 is the seventh coronavirus species that is now known to infect humans, is also zoonotic in origin, and is the causative organism for the current viral pneumonia epidemic in China.


A significantly large variety of coronavirus species cause a diverse range of diseases in domesticated and wild mammals and birds, and these animals may also be carriers of and reservoirs for coronaviruses [2] .


Four species are endemic in human populations, and cause mild common cold symptoms in immunocompetent humans.


The two remaining species, SARS-CoV and MERS-CoV, are zoonotic in origin, and their infection of humans may have fatal outcomes.


Question 10: 

In [149]:
top_n_similar_sentences_glove_cosine(10,30)

QUESTION #10: Tools and studies to monitor phenotypic change and potential adaptation of the virus




['We demonstrate viral genetic and phenotypic differences in viruses from West Africa, which may be relevant to differences in zoonotic potential, highlighting the need for studies of MERS-CoV at the animal-human interface.',
 'New animal models that recapitulate features of human lung disease allow preclinical studies to refine delivery protocols and measure novel metrics of phenotypic correction.',
 'Theoretically, a replicating RNA virus expresses a range of genetic and phenotypic variants and has the potential to generate novel virions, which may be selected in response to environmental pressures.',
 'To assess the risk and potential for human pathogenicity of novel bat viruses, it is crucial to establish valid research tools for comparative in vitro infection modeling.',
 'Future experimental studies on these reassortant viruses, that assess viral transmissibility between species, together with epidemiological studies, such as viral monitoring within Indonesian animal populations 

In [170]:
top_n_similar_sentences_LDA_glove_cosine(10, 5, 30)

QUESTION #10: Tools and studies to monitor phenotypic change and potential adaptation of the virus


Importantly, 149
 Muc4 has been detected in synovial sarcomas in humans, thus presenting a novel tissue Screening genetically diverse mouse models provides an opportunity to identify natural 163 variation in novel factors which drive viral disease responses.


It could facilitate identification of human virus receptors in either computational or experimental studies.


developed an in silico computational framework (P-HIPSTer) that employs structural information to predict more than 280,000 PPIs between 1,001 human-infecting viruses and humans, and made a series of new findings about human-virus interactions (Lasso et al., 2019) .


Daily results for all pathology-related data types (histopathology, weight 329 change, and footpad swelling) was analyzed by three-way ANOVA to determine the impact of 330 mouse strain, sex, and infection status.


In this study, we used TWIRLS, a machine-based approach to collect, summarize, and analyze about 15,000 biomedical articles related to coronavirus, with the aim to elucidate the mechanisms underlying coronavirus-induced host pathological changes.


...

Firstly, the number of human virus receptor proteins was much less than that of human membrane proteins in the modeling, which may hinder accurate modeling.


Largescale and multiple tissue-level analysis of single-cell RNAseq would be more accurate for the expression analysis of ACE2 in different populations.


The genes that serve as linkers are potential targets for gain-and loss-of-function experiments to identify those systems described by the meaningful entities in these categories.


In contrast, the ACE2 expression analysis using the RNA-seq and microarray datasets from control lung tissues indicated there were no significant differences between Asian and Caucasian, or male and female 11 .


Combination of the model with Lasso's work further predicted receptors for 693 human-infecting viruses.


Question 11: 

In [151]:
top_n_similar_sentences_glove_cosine(11,30)

QUESTION #11: Immune response and immunity




['Immune response.',
 'Immune Response.',
 'They confer protection mainly through humoral immune responses with little or no cellular immunity.',
 'Protection against IAV infection is also provided by the humoral immune response.',
 'Activation of innate immunity in response to IFV regulates anti-viral protective immunity 31 .',
 'Activation of innate immunity in response to IFV regulates anti-viral protective immunity 26 .',
 'Apart from the humoral immune response, cell-mediated immune response is also induced after mucosal vaccination.',
 'As the antigens are expressed intracellularly, both humoral and cell-mediated immunity can be activated to offer broad immune protection.',
 'Regardless of any humoral immune response elicited in these animals it is unlikely that antibody alone confers protection.',
 'Evidence indicates that the maternal immune system may tolerate fetal antigens by suppressing cell-mediated immunity while retaining normal humoral immunity.',
 'This result suggests

In [171]:
top_n_similar_sentences_LDA_glove_cosine(11, 5, 30)

QUESTION #11: Immune response and immunity


We saw what appears to be an innate immune response at the 10,000 PFU EBOV exposure level.


It has been suggested that EBOV can mediate an innate immunity response through stimulation of TLR-4 [28] .


Hence, the S2 subunit may serve as an important antigen for inducing 2 0 2 both humoral as well as cell-mediated immunity against SARS-CoV and SARS-CoV-2.


We chose Muc4 192 as our priority candidate gene for follow up, with the initial hypothesis that Muc4 suppressed 193 apoptosis and possible the interferon response, and that its absence in a Muc4 -/mouse would 194 therefore lead to increased apoptosis and inflammation, thereby inhibiting Our validation utilized a Muc4 -/mouse and confirmed a role for Muc4 in protection from 197 SARS-CoV-and CHIKV-induced disease and pathogenesis.


Other vaccine studies have Dosing 
 Most preventative vaccines are designed to elicit a humoral immune response, typically via the administration of whole protein from a pathogen.


...

Due to the nature of RNA virus' rapid mutation rates, changes in the S-protein's amino acid sequence, especially in the RBD's receptor binding motif, could have significant impact on virus infectivity, pathogenicity, transmissibility, and cross-protection from previous coronavirus infection, as well as therapeutic antibody and prophylactic vaccine development.


Neutralizing antibodies targeting epitopes in these regions could potentially have cross-protective activities against different mutant strains.


Although speculative, it is possible that Muc4 functions not to modulate local viral 244 replication, but rather to limit disseminated disease.


It is known that antibodies targeting the S2 region could block S2 protein's configuration change and hence interfere with virus entry into host cells.


The disease was determined to be caused by virus-induced pneumonia by clinicians according to clinical symptoms and other criteria, including a rise in body temperature, decreases in the number of lymphocytes and white blood cells (although levels of the latter were sometimes normal), new pulmonary infiltrates on chest radiography and no obvious improvement after treatment with antibiotics for three days.


Question 12: 

In [153]:
top_n_similar_sentences_glove_cosine(12,30)

QUESTION #12: Effectiveness of movement control strategies to prevent secondary transmission in health care and community settings




['In resource-limited settings, relative to resource-adequate settings, there continues to be a paucity of data in support of infection prevention and control, and patient safety interventions to ensure that regional, if not national, healthcare systems work effectively to improve infection prevention and control interventions.',
 'Thus, studies to prevent transmission in healthcare setting are critical for the development of control measures.',
 'Hence, our current working toolbox available to control the spread of Ebola still hinges on supportive medical care to increase the survival of those infected and basic non-pharmaceutical public health measures [96] to prevent transmission, namely: 1) infection control measures including standard precautions in health care settings; 2) rapid contact tracing and isolation of infectious individuals; and 3) social distancing interventions in the community which may include the dissemination of awareness campaigns to inform the population on how 

In [172]:
top_n_similar_sentences_LDA_glove_cosine(12, 5, 30)

QUESTION #12: Effectiveness of movement control strategies to prevent secondary transmission in health care and community settings


This type of transmission would make effective contact tracing challenging, and good respiratory and hand hygiene would be crucial to reduce this route of transmission, coupled with environmental decontamination in health-care settings.


If COVID-19 can be controlled by isolation and contact tracing, then public health efforts should be focused on this strategy; however, if this is not enough to control outbreaks, then additional resources might be needed for additional interventions.


International investment needs to be directed especially to countries with limited healthcare and public health surveillance capacity to enable the detection of cases and disease control [16, 17] Our estimation showed a lower risk of transmission in Africa and South America.


In this population-level observational study, we used crowdsourced reports from DXY.cn, a social network for Chinese physicians, health-care professionals, pharmacies, and health-care facilities established in 2000.


Future studies based on larger samples of patients with COVID-19 could explore in more detail the transmission dynamics of the outbreak in different locations, the effectiveness of interventions, and the demographic factors driving transmission.


...

The main issue with the quality of patient-level data obtained during health emergencies is the potential lack of information from locations overwhelmed by the outbreak (in this case, Hubei province and other provinces with weaker health infrastructures).


We further looked in more detail at the risk to Africa where the health infrastructure would be challenged tracking a new epidemic across its 54 countries.


The aim of the current study was to explore the effect of sustained transmission from the four Chinese cities of Wuhan, Beijing, Shanghai and Guangzhou on international disease importation risk to 168 countries and territories, with a specific focus on Africa where current levels of healthcare infrastructure could provide a significant challenge for managing this novel epidemic.


Hence countries ranked as high risk in our model (4 th and 3 rd quantiles) should take all steps necessary to ensure prompt detection of cases and the capacity to manage these cases to prevent ongoing spread.


31 Values of the natural history represent the current best understanding of COVID-19 transmission, and we used 20 index cases and a short delay to isolation to represent a relatively large influx into a setting of high awareness of possible infection.


Question 13: 

In [155]:
top_n_similar_sentences_glove_cosine(13,30)

QUESTION #13: Effectiveness of personal protective equipment (PPE) and its usefulness to reduce risk of transmission in health care and community settings, mask, google, gloves




['Effective use of personal protective equipment (PPE) is essential to protect personnel and patients in healthcare settings [1] .',
 'Effective use of personal protective equipment (PPE) by healthcare workers (HCWs) is an important component of infection prevention in healthcare settings.',
 'Third, healthcare providers need to be better trained to maximize the effectiveness of infection control measures, including use of masks, respirators, and other personal protective equipment.',
 'A key aspect of patient isolation is proper use of personal protective equipment (PPE) to protect HCWs from pathogen exposure during patient care.',
 'have shown that in health care settings, the use of masks could reduce the transmission of infl uenza [23] .',
 'Other factors that affect the use of personal protective equipment, such as staff and management attitudes about the value of respirator use, fatigue and the availability of replacement masks, also need to be considered.',
 'Triage, isolation a

In [173]:
top_n_similar_sentences_LDA_glove_cosine(13, 5, 30)

QUESTION #13: Effectiveness of personal protective equipment (PPE) and its usefulness to reduce risk of transmission in health care and community settings


However, there are still many problems with the current psychological interventions, including effective utilization of Internet resources/tools, and efficient cooperation between medical staffs and psychologists.


Whilst strong epidemiology and surveillance systems are indispensable tools for the detection and monitoring of outbreaks and public health emergencies, strong primary care systems form the foundation of any emergency response.


Decent access to primary health care is essential in health emergencies, and its infrastructure crucial for containment, 10 just as good access to high-quality primary care is at the foundation of any strong health system.


As the 'front door' of the health system, primary care professionals should be involved in planning and action for health emergency risk management.


5) is an effective method for understanding and managing psychological impacts among medical staffs, including managing the full risk and resilience in the responder "hazard specific" stress.


...

National primary care bodies can coordinate with public health leads to cascade information to practitioners, communicate with the public, and collate health intelligence from the frontline primary care.


Psychological assistance (such as hotline, online consulting) is used to identify and help the target groups who need intervention.


12 Specifically, psychological crisis interventions should be integrated into the treatment of pneumonia and blocking of the transmission routes.


Moreover, mental health care services for the COVID-19 outbreak are being provided by national hospitals and community mental health centers throughout the country.


WHO has asked all countries to prepare for cases including through surveillance, tracing, treatment and isolation practices, and by sharing data.


Question 14: 

In [157]:
top_n_similar_sentences_glove_cosine(14,30)

QUESTION #14: Role of the environment in transmission




['Uncov- ering the basic mechanisms governing complex human behaviors in resource-poor urban environments is paramount for developing better infrastructure, fostering local economic development and responding to the emergence, transmission and propagation of infectious disease threats.',
 'Thus, studies to prevent transmission in healthcare setting are critical for the development of control measures.',
 'Therefore, improved understanding of the potential wildlife host involvement in the transmission of emerging flaviviruses is essential.',
 'Economic behavior is known to play a key role in disease transmission.',
 'Static and dynamic interactions can play a role in disease transmission [3, 4] .',
 'Human, animal, and vector interactions play a major role in disease transmission and form a dynamic transmission cycle.',
 'These events underscore the potential for aerosol transmission in non-health care settings and the dramatic role such transmission can play in the global transmission 

In [174]:
top_n_similar_sentences_LDA_glove_cosine(14, 5, 30)

QUESTION #14: Role of the environment in transmission


This review does not aspire to cover the large subject of human-to-human transmission control, but also here a mixture of measures is important from strictly medical (transmission routes, efficiency of PPE, vaccines, antivirals and so on) to more social science-oriented (How do people behave when they suspect they could be infected?


There is an urgent need for the implementation of multidisciplinary One Health to address the current complex health challenges at the human-animal-environment interface [42, 43] One Health approaches in China have recently been described [49] [50] [51] [52] .


[3] Studies also showed that the virus has strong human-to-human transmission capability.


The implementation and development of One Health collaborations on a global scale are critical to reduce the threats of emerging viruses [42, 43] .


Fuelled by the availability of new research technologies, as well as changing disease, cost and other pressing issues of our time, further growth in this exciting space will undoubtedly continue.


...

Scientific research is of vital importance for tackling emerging infectious diseases and developing effective intervention methods.


Ominously, a further mathematical model, proposed by Tang et al, [9] suggests that the basic reproductive number for SARS-CoV-2 might be as high as 6.47. outbreak to the global economy will, without doubt, be scrupulously studied after the present outbreak ends, and the global economic costs will be immense, and the human cost, agonizing.


6 These RBPs include tristetraprolin and AUF1, which promote degradation of AU-rich element (ARE)-containing mRNA; members of the Roquin and Regnase families, which respectively promote or effect degradation of mRNAs harbouring stem-loop structures; and the increasingly apparent role of the RNA methylation machinery in controlling inflammatory mRNA stability.


By expanding and cooperatively leveraging our respective research strengths, our efforts may yet solve the many pressing disease, cost and other sustainability issues of our time.


One of the cornerstones and a prerequisite for a proper public health and clinical response is the availability of a reliable diagnostic and reference laboratory service with adequate capacity.
